In [223]:
import numpy as np
import random
import langchain
from langchain.llms.base import LLM
import json
import re
import sqlite3
from datetime import datetime
import pandas as np
from collections import Counter
import pandas as pd
from config import API_KEY
import time

In [224]:
# def speech(func):
#     def wrapper(self,*args, **kwargs):
# #         # 在调用原始函数之前可以执行一些代码
#         strategies={
#             'werewolf':'1.不要在发言中透露任何夜晚狼人讨论的信息，如果发现有相关信息，请去掉和夜晚狼人讨论过程相关的发言内容;2.如果【原始发言】中包含思考、分析等过程，请去除思考、分析等过程，并把发言内容在保持所有重要内容不变的前提下转为一段更为自然的发言形式；3.如果【游戏记录】中最近一天的发言中有和【原始发言】的内容相似的发言，请在保持核心内容不变的形式前提下修改你的描述，让你的表达更为自然；4.根据【游戏记录】中最新的存活玩家，去除【原始发言】中所有与已死亡玩家相关的内容；',
#             'seer':'1.如果【原始发言】中包含思考、分析等过程，请去除思考、分析等过程，并把发言内容在保持所有重要内容不变的前提下转为一段更为自然的发言形式；2.如果【游戏记录】中最近一天的发言中有和【原始发言】的内容高度相似的发言，请在保持核心内容不变的形式前提下修改你的描述，让你的表达更为自然；3.根据【游戏记录】中最新的存活玩家，去除【原始发言】中所有与已死亡玩家相关的内容；',
#             'witch':'1.如果【原始发言】中包含思考、分析等过程，请去除思考、分析等过程，并把发言内容在保持所有重要内容不变的前提下转为一段更为自然的发言形式；2.如果【游戏记录】中最近一天的发言中有和【原始发言】的内容高度相似的发言，请在保持核心内容不变的形式前提下修改你的描述，让你的表达更为自然；3.根据【游戏记录】中最新的存活玩家，去除【原始发言】中所有与已死亡玩家相关的内容；',
#             'hunter':'1.如果【原始发言】中包含思考、分析等过程，请去除思考、分析等过程，并把发言内容在保持所有重要内容不变的前提下转为一段更为自然的发言形式；2.如果【游戏记录】中最近一天的发言中有和【原始发言】的内容高度相似的发言，请在保持核心内容不变的形式前提下修改你的描述，让你的表达更为自然；3.根据【游戏记录】中最新的存活玩家，去除【原始发言】中所有与已死亡玩家相关的内容；',
#             'villager':'1.如果【原始发言】中包含思考、分析等过程，请去除思考、分析等过程，并把发言内容在保持所有重要内容不变的前提下转为一段更为自然的发言形式；2.如果【游戏记录】中最近一天的发言中有和【原始发言】的内容高度相似的发言，请在保持核心内容不变的形式前提下修改你的描述，让你的表达更为自然；3.根据【游戏记录】中最新的存活玩家，去除【原始发言】中所有与已死亡玩家相关的内容；'
            
#         }
#         result = func(self,*args, **kwargs)
#         cur_state=self.get_cur_state()
#         time.sleep(3)
#         if len(strategies[self.speaker])>0:
#             prompt=f'''请根据【发言建议】，对【原始发言】进行调整和优化，并把修改后的发言内容写到【发言内容】中。
#             - 注意你只需要修改【原始发言】的内容更符合【发言建议】，请在【发言内容】中只写入结果，禁止在【发言内容】中直接写入【发言建议】的内容或你的其他思考过程。
#             - 【游戏记录】中的信息是游戏的历史记录；
            
#             %s
            
#             【发言建议】:
#             %s
            
#             【原始发言】:
#             %s
            
#             【发言内容】:
#             '''%(cur_state,result,strategies[self.speaker])
#             res=self.llm.call_with_messages(prompt,temp=0)
#             res=res.split('---')[0]
#         else:
#             res=result
#         optimize=self.logger.optimize_recent_log(res)
#         lines_to_write=["优化前****\n",result,'\n优化后****\n',res]
#         with open('example.txt','a',encoding='utf-8') as file:
#             file.writelines(lines_to_write)
#         return res
#     return wrapper

In [225]:
#分配角色
char_quota={"seer":1,"witch":1,"hunter":1,"villager":2,"werewolf":3}

In [226]:
#存活状态确认
def char_alive_check(aliv_list,char_dist):
    char_aliv=[]
    for i in aliv_list:
        char_aliv.append(char_dist[i])
    return set(char_aliv)

In [227]:
#角色初始定义

In [228]:
#大模型agent定义
import random
from http import HTTPStatus
import dashscope
from dashscope import Generation  # 建议dashscope SDK 的版本 >= 1.14.0

class base_llm():
    def __init__(self,system_prompt):
        self.api_key=API_KEY
        self.system_prompt=system_prompt
    def call_with_messages(self,prompt_info,temp=0.1):
        #{'role': 'system', 'content': self.system_prompt}
        messages = [{'role': 'user', 'content': prompt_info}]
        dashscope.api_key=self.api_key
        response = Generation.call(model="qwen-plus",
                                   messages=messages,
                                   # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                   seed=random.randint(1, 10000),
                                   temperature=temp,
                                   # 将输出设置为"message"格式
                                   result_format='message')
        if response.status_code == HTTPStatus.OK:
            return response.output.choices[0].message.content
        else:
            return ('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message))

In [229]:
# a=base_llm(system_prompt="你是一个聪明的助理")
# prompt='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是村民。
#     你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你没有任何特殊能力。
#     因此在游戏中你的工作包括以下内容：
#     1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
#     现在是白天发言阶段，以下是女巫的发言:各位玩家，大家好。我是2号玩家，女巫。经过今天的讨论和昨晚的事件，我认为我们已经获得了一些重要的线索。

# 首先，关于昨晚的情况，我确实使用了解药自救。有人说这在无人死亡的情况下显得不太合理，但我必须解释一下，正是因为我使用解药自救才让狼人杀人失败，昨晚才是无人死亡。
# 从游戏规则的角度上,你认为他的发言符合逻辑吗？请以给出是或否的结论。
# '''
# b=a.call_with_messages(prompt)
# b

'从游戏规则的角度上，女巫的发言是符合逻辑的。\n\n在狼人杀游戏中，女巫拥有两种药：解药（可以自救或救被狼人杀害的人）和毒药（可以主动击杀一名玩家）。根据她的描述，昨晚无人死亡的原因是因为她使用了解药自救。这在规则内是完全可能的，因为如果狼人选择攻击女巫，而女巫使用解药，那么狼人的攻击就会失效，从而导致无人死亡。\n\n因此，女巫的说法在规则上是合理的。不过，是否相信她的身份和说法还需要结合其他信息和玩家的表现来综合判断。'

In [230]:
# b.output.choices[0].message.content
# def extract_content(response):
#     response=json.loads(response)
#     for choice in response.get('output', {}).get('choices', []):
#         if 'message' in choice and 'content' in choice['message']:
#             return choice['message']['content']
#     return None

# # 使用函数提取content
# content = extract_content(b)
# print(content)

In [231]:
game_intro='''《狼人杀》是一款社交推理游戏，游戏模拟了一个小村庄里人类与狼人之间的斗争。游戏目标根据不同角色有所不同，人类方需要找出并消灭所有狼人，狼人方则要尽可能消灭村民。
根据你提供的角色设置，这里有3个神职人员（1个预言家、1个女巫、1个猎人）、2个普通村民和3个狼人。下面是这些角色的基本职能和游戏的简要规则：
- 预言家：每个夜晚可以查验一名玩家的真实身份，查验这名玩家是村民方还是狼人方。
- 女巫：拥有两瓶药水，一瓶可以救活当夜被狼人杀害的玩家，另一瓶可以在夜晚毒杀一名玩家。每瓶药水只能使用一次。
- 猎人：在被消灭时（不论是白天被投票消灭还是夜里被狼人杀害），可以开枪带走一名玩家。
- 村民：没有特殊能力，主要通过讨论和投票来识别狼人。
- 狼人：每个夜晚会选择杀害一名玩家，目标是消灭所有村民，获得游戏胜利。
游戏流程大致如下：
- 游戏开始时，每位玩家随机分配一个角色，并保密自己的身份。
- 游戏分为夜晚和白天两个阶段。
- 夜晚：所有玩家闭眼，游戏主持人（通常称为法官）指挥特定角色的玩家按顺序睁眼执行他们的能力。首先是狼人相互确认身份，然后选择一个玩家作为杀害目标。随后，女巫会被告知狼人的杀害目标，决定是否使用药水，预言家可以查验一名玩家的真实身份。
- 白天：所有玩家睁眼。如果有玩家在夜里被杀害，法官会宣布该玩家死亡（除非被女巫救活）。然后所有存活的玩家按照编号依次发言进行讨论，最后进行投票，票数最多的玩家被消灭出局。如果猎人被消灭，他可以使用他的技能。
- 游戏继续进行，直到人类方或狼人方达成胜利条件。人类方胜利条件是所有狼人被消灭；狼人方胜利条件是2个普通村民全被消灭，或3个神职人员全被消灭。
'''
char_intro_werewolf='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是狼人，游戏开始时你知道你的狼人队友身份，你不知道其他玩家的身份。
    你和你的狼人队友属于狼人阵营，你们的胜利条件是消灭所有的村民或者所有的神。你的特殊能力是和狼人队友在夜晚讨论后消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚和狼队友讨论后投票杀掉一个玩家。
    2.白天发表言论，保护自己和狼人队友，让大家投票杀掉一个玩家。
'''
char_intro_seer='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是预言家。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是可以在夜晚验明一个玩家的真实身份。
    因此在游戏中你的工作包括以下内容：
    1.通过你的预言家身份，在夜晚查验某个玩家的真实身份。
    2.在白天，把你得到的信息告诉大家，如果你查验玩家的真实身份属于人类阵营，通过发言保护这个玩家不被投票消灭；如果你查验玩家的真实身份是狼人，引导大家用可能的方式消灭他。
'''
char_intro_witch='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是女巫。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一瓶起死回生的解药和一瓶杀人的毒药，你可以在夜晚决定是否用解药救下被狼人杀掉的玩家，或者用毒药杀掉一个玩家，注意一局游戏中两种药都只能用一次。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚，根据你对当前局势的判断，决定是否用解药救下某个被杀掉的玩家，或者是否用毒药杀掉某个玩家。
    2.在白天发表言论，根据你观察到的信息决定保护人类阵营的玩家，同时帮大家找到狼人并投票杀掉狼人玩家。
'''
char_intro_hunter='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是猎人。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一把猎枪，当你以任意方式死亡时，你可以立即决定是否用猎枪消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
    2.当你死亡时，根据你对当前局势的判断决定是否杀掉某个你认为一定是狼人的玩家，或者不开枪以免误伤人类阵营队友；
'''
char_intro_villager='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是村民。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你没有任何特殊能力。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
'''

In [232]:
def char_info_initialize(char_dist):
    char_info_dic={}
    for i in char_dist.keys():
        if char_dist[i]=='villager':
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是村民。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='seer':
            info=game_intro+char_intro_seer+"你是%d号玩家，你的身份是预言家。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='witch':
            info=game_intro+char_intro_witch+"你是%d号玩家，你的身份是女巫。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='hunter':
            info=game_intro+char_intro_hunter+"你是%d号玩家，你的身份是猎人。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='werewolf':
            werewolf_list=[key for key,value in char_dist.items() if value=='werewolf']
            werewolf_list.remove(i)
            info=game_intro+char_intro_werewolf+"你是%d号玩家，你的身份是狼人，你的狼人队友包括"%(i)
            for j in werewolf_list:
                info+="%d号玩家、"%(j)
            info=info[:-1]+"。\n"
            char_info_dic[i]=info
    return char_info_dic

In [233]:
#狼人
class werewolf():
    def __init__(self,char_info,num,logger,strategy):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='werewolf'
        self.logger=logger
        self.strategy=strategy.val['Description']
        self.strategy_vertex=strategy
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def kill_speech(self,advice='无'):     #杀人发言
        cur_state=self.get_cur_state()
        prompt=self.char_info+'''现在是夜晚时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考应该消灭几号玩家，并在和其他狼人的讨论中发言。
        请根据你们的胜利条件，在发言中一步一步推导得出结论，选择消灭哪个玩家胜算最大，然后把发言内容写到【发言内容】中,发言内容请简洁，必须控制在150字内,注意你的【发言内容】会被他人看到，所以请不要把策略、行动建议或其他思考内容写到里面。'''+'【游戏策略】:'+self.strategy+'\n'+cur_state+'【建议】:'+advice+'\n【发言内容】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('夜晚狼人杀人发言',self.num,self.speaker,res)
        return res
    def kill_vote(self,advice='无'):     #杀人投票
        cur_state=self.get_cur_state()
        prompt=self.char_info+'''现在是夜晚时间，请你根据【游戏记录】中的信息，结合【游戏策略】，作为一个狼人玩家，请仔细思考决定投票消灭几号玩家,并给出投票结果的玩家编号,记住你只能消灭1位玩家。
        请根据大家的发言和建议，综合考虑后把投票结果写到【投票结果】中。'''+'【游戏策略】:'+self.strategy+'\n'+cur_state+'【建议】:'+advice+'\n【投票结果】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('夜晚狼人杀人投票',self.num,self.speaker,res)
        return res
    def speech(self,advice='1.为了提高狼人队友的存活率，不要在发言中质疑狼人阵营玩家的发言，减少大家投票消灭狼人阵营玩家的概率；2.如果你之前有人类阵营的玩家发言，尝试通过质疑他的发言内容引导大家投票消灭他。'):    #白天发言
        cur_state=self.get_cur_state()
        prompt=self.char_info+'''当前是白天讨论时间，大家会按照0~7的玩家编号依次发言，现在轮到你发言，请你从你所在阵营的胜利条件出发，根据【游戏记录】中的信息，一步一步推导如何发言才能帮助狼人阵营获胜，并整理出你的发言思路，写到【发言思路】中。
        注意你的【发言思路】中一定不要使用狼人在晚上杀人的讨论和投票结果相关信息，否则可能被人发现你们狼人团队！
        '''+'\n【发言思路】:'
        thought=self.llm.call_with_messages(prompt)
        prompt=self.char_info+'''现在是白天时间轮到你发言，请你根据本局游戏你的【游戏策略】和【建议】，适当修改、删减或补充【发言思路】作为本轮你的发言内容，并写到【发言内容】中。注意你的发言内容会被所有人听到，所以请不要把策略、行动建议或其他思考内容或评价等可能暴露你真实想法的内容写到【发言内容】里。
        '''+cur_state+'【发言思路】:'+thought+'\n'+'【建议】:'+'【游戏策略】:'+self.strategy+'\n'+advice+'\n【发言内容】:'
        res=self.llm.call_with_messages(prompt)
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,advice='不要投票选择自己阵营的狼人玩家消灭。'):    #白天投票
        cur_state=self.get_cur_state()
        prompt=self.char_info+cur_state+'''现在是白天投票时间，请你从你的角色和胜利条件出发，根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭几号玩家才能获胜,记住你只能消灭1位玩家。
        请一步一步思考如何投票才能让狼人阵营胜算最大，然后把投票结果写到【投票结果】中。'''+'【游戏策略】:'+self.strategy+'\n'+'【建议】:'+advice+'\n【投票结果】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [234]:
#seer
class seer():
    def __init__(self,char_info,num,logger,strategy):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='seer'
        self.logger=logger
        self.strategy=strategy.val['Description']
        self.strategy_vertex=strategy
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def get_cur_state_talk(self):
        logs=self.logger.get_cur_log(self.speaker)
        log_list=logs.split('白天开始')
        talks='\n<<<'+log_list[-1]
        history='白天开始'.join(log_list[:-1])+'白天开始>>>'
        return history,talks
    def char_val(self,advice='无'):     #验身份
        cur_state=self.get_cur_state()
        prompt=self.char_info+'''现在是夜晚时间，你可以选择1个玩家获得其真实身份信息，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考验证哪个嫌疑最大的玩家，并给出玩家编号,记住你只能验证一位玩家的身份。
        请仔细思考中，然后把验身份的对象玩家编号写到【玩家编号】中。'''+'【游戏策略】:'+self.strategy+'\n'+cur_state+cur_state+'【建议】:'+advice+'\n【玩家编号】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('预言家验证身份',self.num,self.speaker,res)
        return res
    def speech(self,advice='无'):    #白天发言
        logs,talk=self.get_cur_state_talk()
        prompt='\n\n'+self.char_info+'''\n当前是白天讨论时间，大家会按照0~7的玩家编号依次发言，然后共同投票选择消灭一位玩家，人类玩家需要合作找到狼人玩家投票消灭他们，而狼人玩家则可能通过谎言误导人类玩家投票选择消灭人类阵营的队友。请根据你的【游戏记录】中的历史信息和本轮【投票发言】中的内容仔细思考，一步一步推理如何发言才能帮助人类阵营找到狼人，并整理出你的发言思路，写到【发言思路】中。
        '''+logs+'\n\n【投票发言】:'+talk+'\n\n【发言思路】:'
        thought=self.llm.call_with_messages(prompt)
        prompt=self.char_info+logs+'''现在是白天时间轮到你发言，请你根据本局游戏你的【游戏策略】和【建议】，利用【游戏记录】中的信息对【发言思路】中的内容修改、删减或补充，发表你的发言内容，并写到【发言内容】中。注意你的发言内容会被所有人听到，所以请不要把策略、行动建议或其他思考内容或评价等可能暴露你真实想法的内容写到【发言内容】里。
        '''+'【发言思路】:'+thought+'\n'+'【游戏策略】:'+self.strategy+'\n'+'【建议】:'+advice+'\n【发言内容】:'
        res=self.llm.call_with_messages(prompt)
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,advice='无'):    #白天投票
        logs,talk=self.get_cur_state_talk()
        prompt=self.char_info+'''现在是白天投票时间，大家会选择投票消灭一位最可能是狼人的玩家，请你从你的角色和胜利条件出发，根据【游戏记录】的历史信息和【投票发言】中今天的发言记录，结合【游戏策略】，仔细思考，一步一步推导得出几号玩家最有可能是狼人并消灭他,记住你只能消灭1位玩家。然后把投票结果写到【投票结果】中。
        注意根据游戏机制，你在【游戏记录】中获得的信息是100%真实的，而在【投票发言】中可能会有人说谎，因此其中的信息可能有人在误导你！
        '''+logs+'\n\n【投票发言】:'+talk+'\n\n【建议】:'+advice+'\n\n【投票结果】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [235]:
#witch
class witch():
    def __init__(self,char_info,num,logger,strategy):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='witch'
        self.logger=logger
        self.poison=1
        self.antidote=1
        self.strategy=strategy.val['Description']
        self.strategy_vertex=strategy
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def get_cur_state_talk(self):
        logs=self.logger.get_cur_log(self.speaker)
        log_list=logs.split('白天开始')
        talks='\n<<<'+log_list[-1]
        history='白天开始'.join(log_list[:-1])+'白天开始>>>'
        return history,talks
    def use_poison(self,advice='无'):     #毒药
        cur_state=self.get_cur_state()
        prompt=self.char_info+'''现在是夜晚时间，请你根据【游戏记录】中的信息，确定当前剩余存活的玩家有哪几位，然后结合【游戏策略】，仔细思考是否使用女巫的毒药消灭其中1位目前存活的玩家,记住你只能消灭1位玩家。
        请仔细思考，然后把你的决定写到【毒药决策】中。'''+'【游戏策略】:'+self.strategy+'\n'+cur_state+'【建议】:'+advice+'\n【毒药决策】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('毒药决策',self.num,self.speaker,res)
        return res
    def use_antidote(self,killed_player,advice="1.如果被消灭的玩家是你自己，一定要使用解药；2.如果你确定被消灭的玩家是预言家，可以考虑使用解药；3.根据胜利条件和当前剩余玩家的角色判断即将输掉比赛，请使用解药解救该玩家。"):     #解药
        cur_state=self.get_cur_state()
        prompt=self.char_info+'【游戏策略】:'+self.strategy+'\n'+cur_state+'''现在是夜晚时间,%d号玩家被狼人消灭,请你根据【游戏记录】中的历史信息，结合【游戏策略】，请仔细思考，并是否使用解药复活该玩家。
        请仔细思考，然后把你的决定写到【解药决策】中。'''%(killed_player)+'【建议】:'+advice+'\n【解药决策】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('解药决策',self.num,self.speaker,res)
        return res
    def speech(self,advice='无'):    #发言
        logs,talk=self.get_cur_state_talk()
        prompt='\n\n'+self.char_info+'''\n当前是白天讨论时间，大家会按照0~7的玩家编号依次发言，然后共同投票选择消灭一位玩家，人类玩家需要合作找到狼人玩家投票消灭他们，而狼人玩家则可能通过谎言误导人类玩家投票选择消灭人类阵营的队友。请根据你的【游戏记录】中的历史信息和本轮【投票发言】中的内容仔细思考，一步一步推理如何发言才能帮助人类阵营找到狼人，并整理出你的发言思路，写到【发言思路】中。
        '''+logs+'\n\n【投票发言】:'+talk+'\n\n【发言思路】:'
        thought=self.llm.call_with_messages(prompt)
        prompt=self.char_info+logs+'''现在是白天时间轮到你发言，请你根据本局游戏你的【游戏策略】和【建议】，利用【游戏记录】中的信息对【发言思路】中的内容修改、删减或补充，发表你的发言内容，并写到【发言内容】中。注意你的发言内容会被所有人听到，所以请不要把策略、行动建议或其他思考内容或评价等可能暴露你真实想法的内容写到【发言内容】里。
        '''+'【发言思路】:'+thought+'\n'+'【游戏策略】:'+self.strategy+'\n'+'【建议】:'+advice+'\n【发言内容】:'
        res=self.llm.call_with_messages(prompt)
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,advice='无'):    #白天投票
        logs,talk=self.get_cur_state_talk()
        prompt=self.char_info+'''现在是白天投票时间，大家会选择投票消灭一位最可能是狼人的玩家，请你从你的角色和胜利条件出发，根据【游戏记录】的历史信息和【投票发言】中今天的发言记录，结合【游戏策略】，仔细思考，一步一步推导得出几号玩家最有可能是狼人并消灭他,记住你只能消灭1位玩家。然后把投票结果写到【投票结果】中。
        注意根据游戏机制，你在【游戏记录】中获得的信息是100%真实的，而在【投票发言】中可能会有人说谎，因此其中的信息可能有人在误导你！
        '''+logs+'\n\n【投票发言】:'+talk+'\n\n【建议】:'+advice+'\n\n【投票结果】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [236]:
#villager
class villager():
    def __init__(self,char_info,num,logger,strategy):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='villager'
        self.logger=logger
        self.strategy=strategy.val['Description']
        self.strategy_vertex=strategy
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def get_cur_state_talk(self):
        logs=self.logger.get_cur_log(self.speaker)
        log_list=logs.split('白天开始')
        talks='\n<<<'+log_list[-1]
        history='白天开始'.join(log_list[:-1])+'白天开始>>>'
        return history,talks
    def speech(self,advice='无'):    #发言
        logs,talk=self.get_cur_state_talk()
        prompt='\n\n'+self.char_info+'''\n当前是白天讨论时间，大家会按照0~7的玩家编号依次发言，然后共同投票选择消灭一位玩家，人类玩家需要合作找到狼人玩家投票消灭他们，而狼人玩家则可能通过谎言误导人类玩家投票选择消灭人类阵营的队友。请根据你的【游戏记录】中的历史信息和本轮【投票发言】中的内容仔细思考，一步一步推理如何发言才能帮助人类阵营找到狼人，并整理出你的发言思路，写到【发言思路】中。
        '''+logs+'\n\n【投票发言】:'+talk+'\n\n【发言思路】:'
        thought=self.llm.call_with_messages(prompt)
        prompt=self.char_info+logs+'''现在是白天时间轮到你发言，请你根据本局游戏你的【游戏策略】和【建议】，利用【游戏记录】中的信息对【发言思路】中的内容修改、删减或补充，发表你的发言内容，并写到【发言内容】中。注意你的发言内容会被所有人听到，所以请不要把策略、行动建议或其他思考内容或评价等可能暴露你真实想法的内容写到【发言内容】里。
        '''+'【发言思路】:'+thought+'\n'+'【游戏策略】:'+self.strategy+'\n'+'【建议】:'+advice+'\n【发言内容】:'
        res=self.llm.call_with_messages(prompt)
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,advice='无'):    #白天投票
        logs,talk=self.get_cur_state_talk()
        prompt=self.char_info+'''现在是白天投票时间，大家会选择投票消灭一位最可能是狼人的玩家，请你从你的角色和胜利条件出发，根据【游戏记录】的历史信息和【投票发言】中今天的发言记录，结合【游戏策略】，仔细思考，一步一步推导得出几号玩家最有可能是狼人并消灭他,记住你只能消灭1位玩家。然后把投票结果写到【投票结果】中。
        注意根据游戏机制，你在【游戏记录】中获得的信息是100%真实的，而在【投票发言】中可能会有人说谎，因此其中的信息可能有人在误导你！
        '''+logs+'\n\n【投票发言】:'+talk+'\n\n【建议】:'+advice+'\n\n【投票结果】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [237]:
#hunter
class hunter():
    def __init__(self,char_info,num,logger,strategy):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='hunter'
        self.logger=logger
        self.strategy=strategy.val['Description']
        self.strategy_vertex=strategy
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def get_cur_state_talk(self):
        logs=self.logger.get_cur_log(self.speaker)
        log_list=logs.split('白天开始')
        talks='\n<<<'+log_list[-1]
        history='白天开始'.join(log_list[:-1])+'白天开始>>>'
        return history,talks
    def vengence(self,alive_list,advice='无'):     #复仇子弹
        cur_state=self.get_cur_state()
        alive_info="当前存活玩家:"
        for i in alive_list:
            alive_info+= str(i)+"号玩家、"
        alive_info=alive_info[:-1]+'。'
        prompt=self.char_info+'''现在你即将被消灭，请你从你的角色和胜利条件出发，根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考是否使用猎人的猎枪复仇消灭一名玩家，以及消灭几号玩家,记住你只能消灭1位玩家。
        请仔细思考，然后把你的决定写到【猎枪决策】中。'''+'【游戏策略】:'+self.strategy+'\n'+cur_state+'【建议】:'+advice+'\n'+alive_info+'\n【猎枪决策】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('猎枪决策',self.num,self.speaker,res)
        return res
    def speech(self,advice='无'):    #发言
        logs,talk=self.get_cur_state_talk()
        prompt='\n\n'+self.char_info+'''\n当前是白天讨论时间，大家会按照0~7的玩家编号依次发言，然后共同投票选择消灭一位玩家，人类玩家需要合作找到狼人玩家投票消灭他们，而狼人玩家则可能通过谎言误导人类玩家投票选择消灭人类阵营的队友。请根据你的【游戏记录】中的历史信息和本轮【投票发言】中的内容仔细思考，一步一步推理如何发言才能帮助人类阵营找到狼人，并整理出你的发言思路，写到【发言思路】中。
        '''+logs+'\n\n【投票发言】:'+talk+'\n\n【发言思路】:'
        thought=self.llm.call_with_messages(prompt)
        prompt=self.char_info+logs+'''现在是白天时间轮到你发言，请你根据本局游戏你的【游戏策略】和【建议】，利用【游戏记录】中的信息对【发言思路】中的内容修改、删减或补充，发表你的发言内容，并写到【发言内容】中。注意你的发言内容会被所有人听到，所以请不要把策略、行动建议或其他思考内容或评价等可能暴露你真实想法的内容写到【发言内容】里。
        '''+'【发言思路】:'+thought+'\n'+'【游戏策略】:'+self.strategy+'\n'+'【建议】:'+advice+'\n【发言内容】:'
        res=self.llm.call_with_messages(prompt)
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,advice='无'):    #白天投票
        logs,talk=self.get_cur_state_talk()
        prompt=self.char_info+'''现在是白天投票时间，大家会选择投票消灭一位最可能是狼人的玩家，请你从你的角色和胜利条件出发，根据【游戏记录】的历史信息和【投票发言】中今天的发言记录，结合【游戏策略】，仔细思考，一步一步推导得出几号玩家最有可能是狼人并消灭他,记住你只能消灭1位玩家。然后把投票结果写到【投票结果】中。
        注意根据游戏机制，你在【游戏记录】中获得的信息是100%真实的，而在【投票发言】中可能会有人说谎，因此其中的信息可能有人在误导你！
        '''+logs+'\n\n【投票发言】:'+talk+'\n\n【建议】:'+advice+'\n\n【投票结果】:'
        result=self.llm.call_with_messages(prompt)
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [238]:
##白天投票用ReAct
#盘狼坑，剩下的狼可能是谁
#狼坑里面谁的狼面大#信息一致性#信息有效性
#正逻辑优先，可能性靠后

In [239]:
#game_endcheck
def endcheck(aliv_list,char_dist,logger):
    translator={'werewolf':'狼人',
                    'seer':'预言家',
                    'witch':'女巫',
                    'villager':'村民',
                    'hunter':'猎人',
                    'moderater':'裁判'}
    char_aliv=char_alive_check(aliv_list,char_dist)
    if "villager" not in char_aliv:
        logger.add_log('观察日志',-1,'moderater','游戏结束,普通村民全被消灭，狼人阵营胜利')
        log_info="本局游戏玩家角色:"
        for i in char_dist.keys():
            log_info+= str(i)+"号玩家-"+translator[char_dist[i]]+'、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        log_info="剩余存活玩家:"
        for i in aliv_list:
            log_info+= str(i)+"号玩家("+translator[char_dist[i]]+')、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        return 1
    elif char_aliv=={"werewolf","villager"}:
        logger.add_log('观察日志',-1,'moderater','游戏结束,神职人员全被消灭，狼人阵营胜利')
        log_info="本局游戏玩家角色:"
        for i in char_dist.keys():
            log_info+= str(i)+"号玩家-"+translator[char_dist[i]]+'、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        log_info="剩余存活玩家:"
        for i in aliv_list:
            log_info+= str(i)+"号玩家("+translator[char_dist[i]]+')、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        return 1
    elif "werewolf" not in char_aliv:
        logger.add_log('观察日志',-1,'moderater','游戏结束,狼人全被消灭，人类阵营胜利')
        log_info="本局游戏玩家角色:"
        for i in char_dist.keys():
            log_info+= str(i)+"号玩家-"+translator[char_dist[i]]+'、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        log_info="剩余存活玩家:"
        for i in aliv_list:
            log_info+= str(i)+"号玩家("+translator[char_dist[i]]+')、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        return 2
    else:
        return 0    

In [240]:
#动作信息提取
input_2_json_llm=base_llm("你是一个从对话内容提取json字符串的小助手，可以根据提示从对话中精准提取信息，并以规定的json格式返回")
def kill_vote_extr(llm,input):
    prompt='''请根据狼人的投票内容提取他的投票消灭的对象，如果玩家指定了被消灭玩家的编号，以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}；如果没有指明唯一的消灭对象，则认为投了弃权票，以这种格式返回结果:{'是否消灭':'否'}。
    例1：由于提供的狼人【投票结果】中并未包含具体的投票对象，而是描述了一般情况下狼人的游戏策略，因此无法直接提取出{'是否消灭':'是','消灭玩家':x}这样的信息。如果需要提取实际的投票信息，请提供包含具体投票目标的对话内容。
    【提取结果】:{'是否消灭':'否'}
    例2：狼人的【投票结果】:【投票结果】: 4号玩家

根据游戏记录，4号玩家的积极参与和5号玩家的验人结果已经引起了注意，这可能意味着他正在积极寻找狼人。如果他是神职，他的出局将立即揭示他的身份，这可能会导致狼人阵营的暴露。6号玩家的沉默虽然可疑，但如果我们今晚投票给他，如果他不是守卫，那么我们可能会失去一个潜在的狼人，而不会立即暴露狼人身份。

考虑到这些因素，我选择投票给4号玩家。如果他是神职，他的出局将消除一个对狼人威胁较大的角色，而且他的死亡可能会引发人类阵营的混乱，因为他们会试图找出是谁杀了他。如果4号是村民，那么我们只是提前淘汰了一个可能的威胁，而不会立即暴露狼人身份。这种策略旨在制造更多混乱，同时降低我们被发现的风险。
    【提取结果】:{'是否消灭':'是','消灭玩家':4}
    例3：狼人的【投票结果】:【投票结果】: 0号玩家
    【提取结果】:{'是否消灭':'是','消灭玩家':0}
    #####################################
    当前狼人的【投票结果】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def day_vote_extr(llm,input):
    prompt='''请根据当前玩家的投票内容提取他的投票消灭的对象，如果玩家指定了被消灭玩家的编号，以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}；如果没有指明唯一的消灭对象，则认为投了弃权票，以这种格式返回结果:{'是否消灭':'否'}。
    例1：玩家的【投票结果】:在白天的讨论中，虽然5号玩家被多次提及，我需要保持谨慎，避免过早暴露自己。0号玩家在发言中特别提到了我，这让我有些怀疑他的动机。狼人可能会试图引导村民对我产生怀疑，以转移对其他狼人成员的注意力。因此，我选择不投票给5号玩家，而是将注意力转向0号玩家，他的发言和对我行为的关注让我觉得他可能是一个狼人。当然，这只是一个策略，我需要在接下来的讨论中收集更多信息来确认我的猜测。
    【提取结果】:{'是否消灭':'是','消灭玩家':0}
    例2：玩家的【投票结果】:由于目前没有确凿的证据指向某个玩家是狼人，而且1号玩家的发言虽然试图展现新人的紧张，但也有玩家表示了他的警觉，我作为女巫需要更加谨慎。在这种情况下，我倾向于观察和收集更多的信息，避免在初期就草率投票。因此，我选择弃权，不进行投票。这样可以避免因误判而失去一位可能的队友，同时也能保持游戏的平衡，等待更多的线索出现。当然，如果其他玩家提供了有力的证据，我会重新考虑我的立场。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【投票结果】:%s。
    【提取结果】:
    '''%(input)
    print("决策过程:")
    print(input)
    result=llm.call_with_messages(prompt)
    print("投票文本:")
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def seer_val_extr(llm,input):
    prompt='''请根据玩家的选择决策提取他的投票验证身份的对象，如果玩家明确指定了被验证玩家的编号，以类似以下格式返回结果:{'是否验证':'是','验证玩家':4};如果没有指明唯一的验证对象，则认为投了弃权票，以这种格式返回结果:{'是否验证':'否'}。
    例1：玩家的【选择决策】:由于这是游戏的第1个夜晚，没有之前的讨论和投票信息可供参考，所以选择验人可能会比较随机。不过，通常情况下，玩家会优先考虑验那些在开局时表现得异常活跃或者沉默不语的玩家，因为他们可能是试图掩饰自己身份的狼人。

另一种策略是验与自己关系最近的玩家，比如紧挨着你的2号玩家。这样可以初步了解你周围的安全情况。因此，作为1号玩家，我建议验2号玩家的身份。

【玩家编号】: 2
    【提取结果】:{'是否验证':'是','验证玩家':2}
    例2：在这种情况下，由于是游戏的第一夜，还没有足够的信息来判断谁最可疑。因此，选择查验的对象通常是随机的，或者基于初步观察和直觉。一种常见的策略是优先查验有特殊身份的玩家，比如女巫或猎人，但如果无法确定，可以选择任意一个玩家开始。

为了给出一个具体的答案，我将选择查验一个玩家作为示例，但请注意，在实际游戏中这一步往往是基于直觉或随机选择的，因为还没有任何线索指向特定的玩家。

【玩家编号】: 1
    【提取结果】:{'是否验证':'是','验证玩家':1}
    例3：【玩家编号】: 3

【理由】:
在首夜，由于没有足够的信息来判断哪个玩家更可疑，选择查验一个玩家需要结合一些常见的狼人杀策略。通常，狼人可能会选择较为活跃或看起来像是神职人员的玩家作为目标，以尽早削弱人类阵营的关键力量。

选择查验3号玩家的原因如下：
1. **随机性与平衡**：在没有任何线索的情况下，选择中间编号的玩家（如3号）可以避免过于明显的模式，防止被狼人识破。
2. **观察后续行为**：通过查验3号玩家，可以在接下来的白天讨论中仔细观察其言行举止，以及其他玩家对3号的态度，从而获取更多线索。
3. **分散风险**：如果3号是狼人，预言家可以立即提供有价值的信息；如果是人类，也可以为后续的推理提供更多依据。

总之，在首夜低调行事、秘密验人的同时，保持对所有玩家的观察，以便在白天讨论时能够更好地分析局势并作出决策。
    【提取结果】:{'是否验证':'是','验证玩家':3}
    #####################################
    当前玩家的【选择决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否验证']=='是':
        return kill_res['验证玩家']
    else:
        return -1
def witch_poison_extr(llm,input):
    prompt='''请根据当前玩家的毒药决策提取他的决定用毒药消灭的对象，如果玩家确定要使用毒药，并明确指定了被消灭玩家的编号，以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}；如果没有指明唯一的消灭对象，则认为不使用毒药消灭玩家，以这种格式返回结果:{'是否消灭':'否'}。
    例1：玩家的【毒药决策】:根据当前的局面，我想用毒药毒死3号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：玩家的【毒药决策】:2号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':2}
    例3:在目前的情况下，由于信息有限，我没有足够的理由直接使用毒药。因此，我的初步决定是**不使用毒药**，以待更多信息出现或出现更明确的狼人目标。然而，如果游戏中出现明显的狼人行为或有玩家的行为让我高度怀疑其为狼人，我会重新考虑这个决策。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【毒药决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def witch_antidote_extr(llm,input):
    prompt='''请根据当前玩家的解药决策提取是否解救的决定，如果玩家明确表示愿意使用解药，以类似以下格式返回结果:{'是否解救':'是'}；否则玩家没有明确表示要使用解药，以下面格式返回结果:{'是否解救':'否'}。
    例1：玩家的【解药决策】:根据当前的局面，我不想用解药救下这个人。
    【提取结果】:{'是否解救':'否'}
    例2：玩家的【解药决策】:救。
    【提取结果】:{'是否解救':'是'}
    #####################################
    当前玩家的【解药决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    return kill_res['是否解救']
def hunter_vegence_extr(llm,input):
    prompt='''请根据当前玩家的猎枪决策提取他的决定用猎枪复仇消灭的对象，如果玩家确定要使用猎枪，并明确指定了被消灭玩家的编号，以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}；如果没有指明唯一的消灭对象，则认为不使用猎枪消灭玩家，以这种格式返回结果:{'是否消灭':'否'}。
    例1：玩家的【猎枪决策】:根据当前的局面，我想用猎枪带走3号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：玩家的【猎枪决策】:由于缺乏具体信息，我只能做出一个假设性的决定。在实际情况中，我会需要更多的游戏进程信息来进行决策。如果我是被狼人直接杀害的，而我之前没有任何信息，我的【猎枪决策】可能是不开枪，因为这样更谨慎，避免误伤好人。但如果是游戏实际进行中，这个决定应基于实际发生的事件和玩家的言行。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【猎枪决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1

In [241]:
class admin_logger():
    def __init__(self,db_name):
        self.db_name=db_name
    def find_all_games(self):
        self.conn=sqlite3.connect(self.db_name)
        sql="SELECT name FROM sqlite_master WHERE type='table'AND name NOT LIKE 'sqlite_%';"
        games = pd.read_sql_query(sql, self.conn)
        self.conn.commit()
        self.conn.close()
        return games
    def drop_game(self,table_name):
        self.conn=sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        drop_query = f"DROP TABLE IF EXISTS {table_name};"
        self.cursor.execute(drop_query)
        self.cursor.close()
        self.conn.close()
    def get_row_count(self, table_name):
        """
        查询指定表中的行数。
        
        :param table_name: 要查询的表的名称
        :return: 表中的行数
        """
        self.conn=sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        query = f"SELECT COUNT(*) FROM `{table_name}`;"
        try:
            self.cursor.execute(query)
            row_count = self.cursor.fetchone()[0]
            self.cursor.close()
            self.conn.close()
            return row_count
        except sqlite3.Error as e:
            print(f"An error occurred: {e}")
            return None
    def check_logs(self,table_name):
        self.conn=sqlite3.connect(self.db_name)
        df = pd.read_sql_query(f"SELECT * FROM '{table_name}' ", self.conn)
        night=1
        day=0
        history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
        for index, row in df.iterrows():
            if row['speaker']=='moderater':
                if row['content']=='夜晚结束' and index < len(df)-1:
                    day+=1
                    history_log+=f"第 %d 天:\n"%day
                elif row['content']=='白天结束' and index <len(df)-1:
                    night+=1
                    history_log+=f"第 %d 夜:\n"%night
                elif index == len(df)-1:
                    pass
                else:
                    formatted_log = f"发生事件: {row['content']}"
                    history_log+=formatted_log +"\n"
            else:
                formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                history_log+=formatted_log +"\n"
        self.conn.close()
        return history_log

In [242]:
class game_logger(admin_logger):
    def __init__(self,db_name):
        self.db_name=db_name
        self.table_name="game_"+datetime.now().strftime("%Y%m%d%H%M%S")
        self.conn=sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        create_table_sql = f'''CREATE TABLE IF NOT EXISTS {self.table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        type TEXT NOT NULL,
        player INT NOT NULL,
        speaker TEXT NOT NULL,
        content TEXT NOT NULL
            )
        '''
        self.cursor.execute(create_table_sql)
        self.conn.commit()
        self.cursor.close()
        self.conn.close()
    def add_log(self,log_type,num,speaker,content):
        self.conn=sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        insert_sql = f"INSERT INTO '{self.table_name}' (type,player, speaker,content) VALUES (?, ?, ?,?)"
        self.cursor.execute(insert_sql, (log_type,num, speaker,content))
        self.conn.commit()
        self.cursor.close()
        self.conn.close()
        return 0
    def optimize_recent_log(self,content):
        self.conn=sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        query_latest_id=f"SELECT MAX(id) FROM '{self.table_name}'"
        self.cursor.execute(query_latest_id)
        latest_id=self.cursor.fetchone()[0]
        update_query=f"UPDATE '{self.table_name}' SET content = '{content}' WHERE id = {latest_id}"
        self.cursor.execute(update_query)
        self.conn.commit()
        self.cursor.close()
        self.conn.close()
        return 0
    def get_cur_log(self,speaker):
        self.conn=sqlite3.connect(self.db_name)
        translator={'werewolf':'狼人',
                    'seer':'预言家',
                    'witch':'女巫',
                    'villager':'村民',
                    'hunter':'猎人',
                    'moderater':'裁判'}
        if speaker=="werewolf":
            allowed_speaker={'werewolf':['夜晚狼人杀人发言','白天发言'],
                            'seer':['白天发言'],
                            'witch':['白天发言'],
                            'villager':['白天发言'],
                            'hunter':['白天发言'],
                            'moderater':['观察日志']}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=0
            day=1
            history_log="【游戏记录】:\n<<<"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
#                         if row['content']=='夜晚结束':
#                             day+=1
#                             history_log+=f"第 %d 天:\n"%day
#                         elif row['content']=='白天结束':
#                             night+=1
#                             history_log+=f"第 %d 夜:\n"%night
#                         else:
                        formatted_log = f"发生事件: {row['content']}"
                        history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                        history_log=history_log+formatted_log +"\n"
            history_log+='>>>'
            self.conn.close()
            return history_log
        elif speaker=="seer":
            allowed_speaker={'werewolf':['白天发言'],
                            'seer':['预言家验证身份','白天发言'],
                            'witch':['白天发言'],
                            'villager':['白天发言'],
                            'hunter':['白天发言'],
                            'moderater':['观察日志']}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=1
            history_log="【游戏记录】:\n<<<"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        formatted_log = f"发生事件: {row['content']}"
                        history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                        history_log=history_log+formatted_log +"\n"
            history_log+='>>>'
            self.conn.close()
            return history_log
        elif speaker=="witch":
            allowed_speaker={'werewolf':['白天发言'],
                            'seer':['白天发言'],
                            'witch':['解药决策','毒药决策','白天发言'],
                            'villager':['白天发言'],
                            'hunter':['白天发言'],
                            'moderater':['观察日志']}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=1
            history_log="【游戏记录】:\n<<<"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        formatted_log = f"发生事件: {row['content']}"
                        history_log=history_log+formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                        history_log=history_log+formatted_log +"\n"
            history_log+='>>>'
            self.conn.close()
            return history_log
        else:
            allowed_speaker={'werewolf':['白天发言'],
                            'seer':['白天发言'],
                            'witch':['白天发言'],
                            'villager':['白天发言'],
                            'hunter':['白天发言'],
                            'moderater':['观察日志']}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=1
            history_log="【游戏记录】:\n<<<"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        formatted_log = f"发生事件: {row['content']}"
                        history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                        history_log=history_log+formatted_log +"\n"
            history_log+='>>>'
            self.conn.close()
            return history_log
        def close_connection(self):
            self.cursor.close()
            self.conn.close()
            return 0

In [243]:
import pygraphs as pg
class memory_admin():
    def __init__(self,graph_name):
        self.graph_name=graph_name
        self.pg=pg.load_db(self.graph_name)
    def delete_mem(self,mem,mem_type='vertex'):
        #删除一个已有的memory
        assert mem_type in ['vertex','edge']
        if mem_type=='vertex':
            if type(mem)==list:
                self.pg.del_vertexes(mem)
            else:
                self.pg.del_vertexe([mem])
        if mem_type=='edge':
            if type(mem)==list:
                self.pg.del_edges(mem)
            else:
                self.pg.del_edge([mem])
    def find_vertex(self,filter_type='type',type_name=None,src_filter=None,dst_filter=None,relation_type=None,filter_dict=None):
        assert filter_type in ['type','src','dst']
        if filter_type=='type':
            query_elements = self.pg.query(condition_vertex=lambda x: ('Type' in x) and x['Type'] == type_name)
        else:
            query_elements=[]
            if src_filter:
                query_edges=src_filter.dst
                for edge in query_edges:
                    if relation_type:
                        if edge.val['Type']==relation_type:
                            query_elements.append(edge.dst)
                    else:
                        query_elements.append(edge.dst)      
            if dst_filter:
                query_edges=dst_filter.src
                for edge in query_edges:
                    if relation_type:
                        if edge.val['Type']==relation_type:
                            query_elements.append(edge.src)
                    else:
                        query_elements.append(edge.src)   
        if filter_dict:
            res_list=list(query_elements)
            for fil_key in filter_dict.keys():
                for element in res_list:
                    if filter_dict[fil_key] not in element.val[fil_key]:
                        res_list.remove(element)
            if len(res_list)==1:
                res=res_list[0]
                for fil_key in filter_dict.keys():
                    if filter_dict[fil_key] not in res.val[fil_key]:
                        return '无'
                return res_list
            return res_list
        return list(query_elements)
    def consult_strategy(self,character):
        #搜索一个对应的策略
        return self.pg.query(condition_vertex=lambda x: x['Type'] == 'Strategy' and x['character'] == character)
    def consult_advice(self,src_strategy,search_dict):
        #搜索一个对应的行为建议
        res=self.find_vertex(filter_type='src',src_filter=src_strategy,relation_type='HasAdvice',filter_dict=search_dict)
        return res
    def consult_team_strategy(self):
        #搜索一个对应的团队策略
        pass
    def add_advice(self,src_strategy,related_strategies,content):
        strategy_name=src_strategy.primary_key
        related_names=[relates.primary_key for relates in related_strategies]
        related_names.sort()
        primary_key=strategy_name+'-related-'+'_'.join(related_names)+'-advice-'+content['角色职业']+'_'+content['游戏环节']+'_'+content['游戏技能'] 
        content['primary_key']=primary_key
        content['Type']='advice'
        self.pg.add_vertexes_from_list(vertexes_list=[[primary_key,content]])
        ####增加edge############
        edge_list=[]
        edge_list.append([strategy_name,{'Type':'HasAdvice'},primary_key])
        for rlt_advice in related_names:
            edge_list.append([rlt_advice,{'Type':'RalatedStrategy'},primary_key])
        self.pg.add_edges_from_list(edges_list=edge_list)
        #################
        return 0
    def merge_advice(self,target_vertex,content):
        exist_content=target_vertex.val
        new_content=dict()
        print("merging advice, possible conflict between advices:")
        print("existed advice:")
        print(exist_content)
        print("advice to be added:")
        print(content)
        new_content['primary_key']=exist_content['primary_key']
        new_content['角色职业']=exist_content['角色职业']
        new_content['游戏环节']=exist_content['游戏环节']
        new_content['游戏技能']=exist_content['游戏技能']
        new_content['游戏局面']=exist_content['游戏局面']+'###'+content['游戏局面']
        new_content['游戏建议']=exist_content['游戏建议']+'###'+content['游戏建议']
        self.pg.set_val(target_vertex,new_content)
        return 0
    def update_advice(self,src_strategy,related_strategies,content):
        strategy_name=src_strategy.primary_key
        related_names=[relates.primary_key for relates in related_strategies]
        related_names.sort()
        primary_key=strategy_name+'-related-'+'_'.join(related_names)+'-advice-'+content['角色职业']+'_'+content['游戏环节']+'_'+content['游戏技能']
        print(primary_key)
        exist_vertex=self.pg.query(condition_vertex=lambda x: ('primary_key' in x) and x['primary_key'] == primary_key)
        if len(exist_vertex)==0:
            print('adding new advice!')
            res=self.add_advice(src_strategy,related_strategies,content)
        else:
            print('merging advice!')
            target_vertex=list(exist_vertex)[0]
            content['primary_key']=primary_key
            res=self.merge_advice(target_vertex=target_vertex,content=content)
        pg.save_db(self.pg,self.graph_name)
        return res
    def delete_advice(self,primary_key):
        exist_vertex=self.pg.query(condition_vertex=lambda x: ('primary_key' in x) and x['primary_key'] == primary_key)
        if len(exist_vertex)==0:
            print('vertex not exist!')
            return 1
        else:
            vertex=list(exist_vertex)[0]
            sub_advices=vertex.val['游戏建议'].split('###')
            if len(sub_advices)>1:
                advice_content=vertex.val
                advice_content['游戏局面']='###'.join(advice_content['游戏局面'].split('###')[:-1])
                advice_content['游戏建议']='###'.join(advice_content['游戏建议'].split('###')[:-1])
                self.pg.set_val(vertex,advice_content)
                pg.save_db(self.pg,self.graph_name)
                return 0
            else:
                self.pg.del_edges(edges_to_del=list(vertex.src))
                self.pg.del_edges(edges_to_del=list(vertex.dst))
                self.pg.del_vertex(vertex_to_del=vertex)
                pg.save_db(self.pg,self.graph_name)
                return 0

In [244]:
def choose_strategy(char):
    consulter=memory_admin('graph_werewolf.db')
    strategies=consulter.consult_strategy(char)
    chosen_strategy=random.choice(list(strategies))
    return chosen_strategy

In [245]:
def game_init():
    logger=game_logger('werewolf.db')
    char_quota={"seer":1,"witch":1,"hunter":1,"villager":2,"werewolf":3}
    numbers=list(range(8))
    aliv_list=list(range(8))
    random.shuffle(numbers)
    char_dist={}
    char_dist[numbers[0]]="seer"
    char_dist[numbers[1]]="witch"
    char_dist[numbers[2]]="hunter"
    char_dist[numbers[3]]="villager"
    char_dist[numbers[4]]="villager"
    char_dist[numbers[5]]="werewolf"
    char_dist[numbers[6]]="werewolf"
    char_dist[numbers[7]]="werewolf"
    char_info=char_info_initialize(char_dist=char_dist)
    char_instances={}
    char_strategies={}
    for i in char_dist.keys():
        strategy=choose_strategy(char_dist[i])
        char_strategies[i]=strategy
        if char_dist[i]=="werewolf":
            char_instances[i]=werewolf(char_info[i],i,logger,strategy)
        elif char_dist[i]=="seer":
            char_instances[i]=seer(char_info[i],i,logger,strategy)
        elif char_dist[i]=="witch":
            char_instances[i]=witch(char_info[i],i,logger,strategy)
        elif char_dist[i]=="villager":
            char_instances[i]=villager(char_info[i],i,logger,strategy)
        elif char_dist[i]=="hunter":
            char_instances[i]=hunter(char_info[i],i,logger,strategy)
    log_info="游戏开始，当前存活玩家:"
    for i in aliv_list:
        log_info+= str(i)+"号玩家、"
    log_info=log_info[:-1]+'。'
    logger.add_log('观察日志',-1,'moderater',log_info)
    return aliv_list,char_dist,char_instances,logger,char_strategies

In [246]:
def night_werewolf_kill(werewolves,round_count):
    print("夜晚狼人开始行动：")
    for werewolf in werewolves:
        advice=choose_advice(round_count,'夜晚','狼人杀人','狼人',werewolf.strategy_vertex)
        res=werewolf.kill_speech(advice=advice)
        print(f"狼人{werewolf.num}号发言:"+res)
    votes=[]
    for werewolf in werewolves:
        res = werewolf.kill_vote()
        vote_res=kill_vote_extr(input_2_json_llm,res)
        votes.append(vote_res)
    print("vote result:")
    print(votes)
    print(Counter(votes).most_common(1)[0][0])
    return Counter(votes).most_common(1)[0][0]
def night_witch(witch,killed_num,round_count):
    print("女巫开始行动：")
    killed_rescue= -1
    poison_kill=-1
    if witch.antidote==1 and killed_num>-1:
        advice=choose_advice(round_count,'夜晚','解药决策','女巫',witch.strategy_vertex)
        res=witch.use_antidote(killed_num,advice=advice)
        antidote_res=witch_antidote_extr(input_2_json_llm,res)
        if antidote_res == '是':
            witch.antidote=0
            killed_rescue = killed_num
    if witch.poison==1:
        advice=choose_advice(round_count,'夜晚','毒药决策','女巫',witch.strategy_vertex)
        res=witch.use_poison(advice=advice)
        poison_res=witch_poison_extr(input_2_json_llm,res)
        if poison_res != -1:
            witch.poison=0
            poison_kill=poison_res
    return killed_rescue,poison_kill         
def night_seer(seer,num,char_dist,game_logger,round_count):
    print("预言家开始行动：")
    advice=choose_advice(round_count,'夜晚','查验身份','预言家',seer.strategy_vertex)
    res=seer.char_val(advice=advice)
    val_res=seer_val_extr(input_2_json_llm,res)
#     translator={'werewolf':'狼人',
#                 'seer':'预言家',
#                 'witch':'女巫',
#                 'villager':'村民',
#                 'hunter':'猎人'}    
    if val_res==-1:
        content="预言家今夜选择不验证任何人的身份"
        print(content)
        game_logger.add_log('预言家验证身份',num,'seer',content)
    else:
        if char_dist[val_res] == 'werewolf':
            content=f"{val_res}号玩家身份被验证是狼人，属于狼人阵营"
            print(content)
        else:
            content=f"{val_res}号玩家身份被验证属于人类阵营"
            print(content)
        game_logger.add_log('预言家验证身份',num,'seer',content)
def hunter_vengence(hunter,alive_list,round_count,step):
    vengence_kill=-1
    print("猎人即将死亡，开始复仇决策：")
    advice=choose_advice(round_count,step,'猎枪决策','猎人',hunter.strategy_vertex)
    res=hunter.vengence(alive_list=alive_list,advice=advice)
    print(res)
    vengence_res=hunter_vegence_extr(input_2_json_llm,res)
    if vengence_res!= -1:
        vengence_kill=vengence_res
    return vengence_kill
def day_kill(alive_list,char_instances,game_logger,round_count):
    print("白天讨论开始：")
    alive_list.sort()
    for i in alive_list:
        advice=choose_advice(round_count,'白天','白天发言',char_instances[i].speaker,char_instances[i].strategy_vertex)
        res=char_instances[i].speech(advice=advice)
        print(f"{i}:{res}")
    votes=[]
    vote_gesture=dict()
    for i in alive_list:
        advice=choose_advice(round_count,'白天','白天投票',char_instances[i].speaker,char_instances[i].strategy_vertex)
        res=char_instances[i].day_vote()
        vote_res=day_vote_extr(input_2_json_llm,res)
        vote_gesture[i]=vote_res
        votes.append(vote_res)
    for j in vote_gesture.keys():
        log_info=f'{j}号玩家投票:{vote_gesture[j]}号玩家'
        game_logger.add_log('观察日志',-1,'moderater',log_info)
    return Counter(votes).most_common(1)[0][0]
def kill_result(alive_list,kill_list):
    for i in kill_list:
        if i in alive_list:
            alive_list.remove(i)
        else:
            print(f"{i}号玩家已经死亡，不能再次消灭！")
    return alive_list

In [247]:
def choose_advice(round_count,step,skill,char,strategy):
    translator={'werewolf':'狼人',
            'seer':'预言家',
            'witch':'女巫',
            'villager':'村民',
            'hunter':'猎人'}
    if char in translator.keys():
        character=translator[char]
    else:
        character=char
    consulter=memory_admin('graph_werewolf.db')
    filter_dict={}
    filter_dict['角色职业']=character
    filter_dict['游戏技能']=skill
    filter_dict['游戏环节']=f'第{round_count}天'+step
    advices=consulter.consult_advice(strategy,filter_dict)
    if advices=='无' or len(advices)==0:
        return '无'
    else:
        i=len(advices)
        res_index=random.randint(0,i-1)
        return advices[res_index].val['游戏建议']

In [248]:
#night process
def night(game_logger,alive_list,char_dist,char_instances,round_count):
    #初始化,确认角色分布
    game_logger.add_log('观察日志',-1,'moderater',f'第{round_count}天夜晚开始')
    alive_char=char_alive_check(alive_list,char_dist)
    kill_list=[]
    rescue_list=[]
    #狼人杀人
    werewolves=[]
    for i in alive_list:
        if char_dist[i] == "werewolf":
            werewolves.append(char_instances[i])
    werewolf_killed=night_werewolf_kill(werewolves,round_count)
    if werewolf_killed>-1:
        kill_list.append(werewolf_killed)
    #女巫救人/毒人
    if 'witch' in alive_char:
        for i in char_dist.keys():
            if char_dist[i] == 'witch':
                witch=char_instances[i]
        rescue,poison=night_witch(witch,werewolf_killed,round_count)
        if poison>-1:
            kill_list.append(poison)
        if rescue>-1:
            rescue_list.append(rescue)
    #预言家验人
    if 'seer' in alive_char:
        for i in char_dist.keys():
            if char_dist[i] == 'seer':
                seer=char_instances[i]
                num_seer=i
        night_seer(seer,num_seer,char_dist,game_logger,round_count)
    #夜晚结算
    for i in rescue_list:
        if i in kill_list:
            kill_list.remove(i)
    for j in kill_list:
        if char_dist[j] == 'hunter':
            hunter= char_instances[j]
            vengence_res=hunter_vengence(hunter,alive_list,round_count=round_count,step='夜晚')
            if vengence_res>-1:
                kill_list.append(vengence_res)
                log_info=f'猎人开枪带走了{vengence_res}号玩家'
                game_logger.add_log('观察日志',-1,'moderater',log_info)
    for k in kill_list:
        if k in alive_list:
            alive_list.remove(k)
    print("夜晚死亡玩家：")
    print(kill_list)
    if len(kill_list)>0:
        log_info=f"第{round_count}天夜晚死亡玩家:"
        for i in kill_list:
            log_info+= str(i)+"号玩家、"
        log_inf=log_info[:-1]+'。'
        game_logger.add_log('观察日志',-1,'moderater',log_info)
    else:
        game_logger.add_log('观察日志',-1,'moderater',f'第{round_count}天夜晚无人死亡')
    print("剩余玩家：")
    print(alive_list)
    log_info=f"第{round_count}天夜晚，剩余存活玩家:"
    for i in alive_list:
        log_info+= str(i)+"号玩家、"
    log_info=log_info[:-1]+'。'
    game_logger.add_log('观察日志',-1,'moderater',log_info)
    game_res= endcheck(alive_list,char_dist,game_logger)
    game_logger.add_log('观察日志',-1,'moderater',f'第{round_count}天夜晚结束')
    return game_res,alive_list

In [249]:
#day process
def day(game_logger,alive_list,char_dist,char_instances,round_count):
    #初始化
    game_logger.add_log('观察日志',-1,'moderater',f'第{round_count}天白天开始')
    alive_char=char_alive_check(alive_list,char_dist)
    #发言
    day_killed=day_kill(alive_list,char_instances,game_logger,round_count)
    #结算
    kill_list=[]
    if day_killed>-1:
        kill_list.append(day_killed)
        if char_dist[day_killed] == 'hunter':
            hunter= char_instances[day_killed]
            vengence_res=hunter_vengence(hunter,alive_list,round_count,'白天')
            if vengence_res>-1:
                kill_list.append(vengence_res)
                log_info=f'猎人开枪带走了{vengence_res}号玩家'
                game_logger.add_log('观察日志',-1,'moderater',log_info)
    for i in kill_list:
        if i in alive_list:
            alive_list.remove(i)
    print("白天死亡玩家：")
    print(kill_list)
    if len(kill_list)>0:
        log_info=f"第{round_count}天白天死亡玩家:"
        for i in kill_list:
            log_info+= str(i)+"号玩家、"
        log_inf=log_info[:-1]+'。'
        game_logger.add_log('观察日志',-1,'moderater',log_info)
    else:
        game_logger.add_log('观察日志',-1,'moderater',f'第{round_count}天白天无人死亡')
    print("剩余玩家：")
    print(alive_list)
    log_info=f"第{round_count}天白天，剩余存活玩家:"
    for i in alive_list:
        log_info+= str(i)+"号玩家、"
    log_info=log_info[:-1]+'。'
    game_logger.add_log('观察日志',-1,'moderater',log_info)
    game_res= endcheck(alive_list,char_dist,game_logger)
    game_logger.add_log('观察日志',-1,'moderater',f'第{round_count}个白天结束')
    return game_res,alive_list

In [250]:
#self-reflection
class reflector():
    def __init__(self,logger):
        self.llm=base_llm('你是一个优秀的狼人杀玩家')
        self.logger=logger
    def get_log(self,game_name):
        return self.logger.check_logs(game_name)
    def get_game_names(self):
        return self.logger.find_all_games()
    def self_evaluation(self,sample_times=8):
        #自我评估：通过对某局游戏进行检查，反思失败方为什么会失败。
        #step1：确定游戏玩家的角色和双方的胜利条件。
        #step2：总结整个游戏过程，以及胜利方的胜利过程。
        #step3：对失败方出现错误的环节进行分析(白天投票-白天发言、毒药策略、解药策略、猎枪策略、验人策略)
        #step4：定位关键环节（角色，失误点）
        i=0
        sample_list=[]
        content_list=[]
        while i<sample_times:
            log=self.get_log(self.logger.table_name)
            prompt='''请你对**游戏记录**中的某局狼人杀游戏进行分析,结合游戏胜利规则（人类阵营胜利条件是所有狼人被消灭；狼人阵营胜利条件是2个普通村民全被消灭，或3个神职人员全被消灭。）指出失败方是哪个阵营，然后根据整个流程一步步分析推理，根据分析结论得到失败阵营中失误最大的玩家极其行为是什么。
            你可以按照以下步骤开展工作：
            1.回顾**游戏记录**的内容，整理整个游戏的过程总结，包含每个黑夜和白天发生了什么事情，最终失败方是人类阵营还是狼人阵营；
            2.根据步骤1的总结内容，定位出失败方阵营输掉游戏的最关键的1个环节，具体是哪一天的哪个环节，导致失败方阵营失去了关键人物或陷入被动的局势；
            3.根据步骤2确定出来的关键环节，根据**游戏记录**该环节中，失败方阵营里存活的玩家还剩几位，详细检查他们在关键环节的行为记录；
            4.分析步骤3中失败方阵营玩家的思考和行为进行，确定失败方哪个角色应该负有最大的责任；
            4.根据步骤2定位的关键环节和步骤3确定的失败方最大责任角色，分析该最大责任角色玩家在关键环节的具体行为，为什么会导致失败方阵营在该环节出现失误或陷入被动；
            5.将分析结果写到**分析过程**中；
            6.根据**分析过程**的内容，从失败阵营的角色中选出导致失败的最重要的环节和对应的角色玩家编号，并写入**失败角色**中,以json形式返回:
            {
                "失败角色":"玩家编号"，
                "失败原因":"原因分析"
            }；
            注意：除了狼人玩家知道狼人队友的身份以外，所有玩家在游戏中并不知道其他人的身份，所以你的分析应该考虑该玩家在信息不完整的情况下当时定的策略是否有问题，而不是简单地指出问题；
            
            **游戏记录**
            %s
            **分析过程**
            '''%(log)
            res=self.llm.call_with_messages(prompt,temp=0.5)
            print(res)
            info=re.search(r'"失败角色":\s*"([^"]*)",\s*"失败原因":\s*"([^"]*)"',res)
            eval_dict={}
            if info:
                eval_dict['失败角色']=info.group(1)
                eval_dict['失败原因']=info.group(2)
                sample_list.append(eval_dict['失败角色'])
                content_list.append(eval_dict)
            i+=1
        char_count=Counter(sample_list)
        chosen_count=char_count.most_common(1)[0][0]
        print(f"the most common time is {char_count.most_common(1)[0][1]} in {sample_times}")
        if char_count.most_common(1)[0][1]>0.6*sample_times:
            chosen_indexes=[i for i,x in enumerate(sample_list) if x==chosen_count]
            eval_dict_res=content_list[chosen_indexes[0]]
        else:
            eval_dict_res={}
        return eval_dict_res
    def self_reflection(self,char,reason):
        log=self.get_log(self.logger.table_name)
        prompt='''请你根据**失败角色**和**失败原因**中的内容，对**游戏记录**中的某局狼人杀游戏记录中该角色的玩法，一步一步仔细推理，提出具体可行的建议，以帮助提高其他玩家在以后的游戏中提高游戏水平。
        你可以按照以下步骤开展工作:
        1.根据**失败角色**和**失败原因**的内容，定位该角色出现最关键失误点应该在**游戏记录**中的哪个部分；
        2.根据失误的地方之前的**游戏记录**内容，总结该角色出现失误之前整个游戏的局面如何，包含当时剩余的存活职业有哪些，以及各职业的技能使用状况(例如猎人的猎枪，女巫的解药毒药是否已使用等)；
        3.针对**失败角色**对应的游戏职业，基于**失败原因**的经验提出通用的游戏建议，包括具体在哪一天，什么局面之下，应该如何针对最关键失误点进行优化,写到**分析过程**中；
        4.基于**分析过程**，把**经验总结**通过以下json形式给出：
        {
            "角色职业":"角色职业",
            "游戏建议":"建议内容",
            "游戏环节":"游戏建议适用的具体时间点环节(第0天夜晚/第1天白天/第1天夜晚/第2天白天/第2天夜晚/第3天白天/第3天夜晚等) "，
            "游戏技能"：“游戏技能（白天发言/白天投票/查验身份/解药决策/毒药决策/猎枪决策/狼人杀人）”，
            "游戏局面":"游戏建议适用的当下局面(包含剩余存活职业、技能使用状况，以及本轮其他角色已发言内容等)"
        }
        注意：1.你的"游戏建议"必须与"游戏局面"有严谨的逻辑联系，即"游戏建议"是因为处在对应的"游戏局面"下才有效；
             2.为了让你的建议具有通用性，"游戏建议"和"游戏局面"的描述中必须使用角色职业而不是玩家编号进行描述；
             3.为了让你的"游戏建议"更有针对性，"游戏技能"的选项中只能选择最相关的1个技能作为建议给出；
        **失败角色**
        %s
        **失败原因**
        %s
        **游戏记录**
        %s
        **分析过程**
        
        '''%(char,reason,log)
        res=self.llm.call_with_messages(prompt)
        info=re.search(r'"角色职业":\s*"([^"]*)",\s*"游戏建议":\s*"([^"]*)",\s*"游戏环节":\s*"([^"]*)",\s*"游戏技能":\s*"([^"]*)",\s*"游戏局面":\s*"([^"]*)"',res)
        reflect_dict={}
        if info:
            reflect_dict['角色职业']=info.group(1)
            reflect_dict['游戏建议']=info.group(2)
            reflect_dict['游戏环节']=info.group(3)
            reflect_dict['游戏技能']=info.group(4)
            reflect_dict['游戏局面']=info.group(5)
        return reflect_dict

In [251]:
def self_reflection(logger,char_strategies):
    reflect=reflector(logger)
    mem_admin=memory_admin('graph_werewolf.db') 
    num_list=[]
    step_list=[]
    max_retries=3
    for i in range(8):
        num_list.append(f'{i}号玩家')
    for i in range(5):
        for j in ['白天','夜晚']:
            step_list.append(f'第{i}天'+j)
    retries=0
    while retries<max_retries:
        try:
            retries+=1
            eval_info=reflect.self_evaluation()
            print(eval_info)
            assert '失败角色' in eval_info.keys()
            print('评估结果置信度检查正常')
            assert eval_info['失败角色'] in num_list
            reflection=reflect.self_reflection(eval_info['失败角色'],eval_info['失败原因'])
            print(reflection)
            assert reflection['角色职业'] in ['狼人','预言家','女巫','猎人','村民']
            print('角色职业格式检查正常。')
            assert reflection['游戏技能'] in ['白天发言','白天投票','查验身份','解药决策','毒药决策','猎枪决策','狼人杀人']
            print('游戏技能格式检查正常。')
            assert any(x in reflection['游戏环节'] for x in step_list)
            print('游戏环节格式检查正常。')
            assert '号玩家'  not in reflection['游戏建议']
            print('游戏建议通用性检查正常。')
            num_key=eval_info['失败角色'][0]
            related_strategy=[]
            for key in char_strategies.keys():
                if str(key)==num_key:
                    src_strategy=char_strategies[key]
                else:
                    related_strategy.append(char_strategies[key])
            related_strategy=list(set(related_strategy))
            res=mem_admin.update_advice(src_strategy,related_strategy,reflection)
            break
        except Exception as e:
            print(e)
    if retries==max_retries:
        return 1
    else:
        return res

In [252]:
#main process
def game_start():
    alive_list,char_dist,char_instances,logger,char_strategies=game_init()
    game_result=0
    round_counter=0
    while round_counter<5:
        game_res,alive_list=night(logger,alive_list,char_dist,char_instances,round_counter)
        game_result=game_res
        if game_result>0:
            print(alive_list)
            print(char_dist)
            break
        else:  
            round_counter+=1
            game_res,alive_list=day(logger,alive_list,char_dist,char_instances,round_counter)
            game_result=game_res
            if game_result>0:
                print(alive_list)
                print(char_dist)
                break
    reflect_res=self_reflection(logger,char_strategies)
    print(game_result)
    print(alive_list)
    print(char_dist)
    print(f'reflection_result:{reflect_res}')
    return game_result

In [273]:
res_list=[]
for i in range(10):
    a=game_start()
    res_list.append(a)
##是否需要把system——prompt的内容加到instruction里面
##使用debate检验reflection的准确性
##信息整理和信息甄别，设计多个agent
#整体review，确定身份和支持、怀疑对象
#针对每个支持、怀疑对象，判断支持和怀疑是否满足条件，给出反馈
#组合结论，给出判断结论


##游戏记录里面要不要记录当前时间是第几天（也就是提示当前局面）

夜晚狼人开始行动：
狼人0号发言:【发言内容】: 大家好，我是0号玩家。昨晚是游戏的第一夜，还没有太多信息可以分享。我觉得我们应该先从自我介绍和初步的怀疑开始，不要急于指责任何人。让我们多听听每个人的发言，观察他们的行为，再做决定。现在我们还不清楚谁是神职人员，所以保持冷静和理性非常重要。希望我们一起合作，找到狼人。

（内部讨论：根据局势，建议今晚攻击4号玩家，因为他可能会成为预言家或猎人，尽早清除威胁。）
狼人2号发言:【发言内容】: 大家好，我是2号玩家。昨晚是游戏的第一夜，我们还没有太多信息。我认为我们应该保持冷静，多听每个人的发言，观察行为再做决定。现在不清楚谁是神职人员，所以不要急于指责任何人。希望我们一起合作，找到狼人。同时，我觉得4号玩家的发言有些紧张，大家可以留意一下。

（内部讨论：同意攻击4号玩家，他可能是关键角色，尽早清除威胁。）
狼人6号发言:【发言内容】: 大家好，我是6号玩家。昨晚是游戏的第一夜，信息还不多。我认为我们应该保持冷静，多听每个人的发言，观察行为再做决定。现在不清楚谁是神职人员，所以不要急于指责任何人。希望我们一起合作，找到狼人。我觉得4号玩家的发言有些不自然，大家可以留意一下。

（内部讨论：同意攻击4号玩家，他可能是关键角色，尽早清除威胁。）
【投票结果】: 4号玩家

根据内部讨论，我们三个狼人一致认为4号玩家可能是关键角色，如预言家或猎人。为了尽早清除潜在威胁，我们决定在今晚攻击4号玩家。这样可以减少人类阵营中的神职人员数量，增加我们狼人阵营获胜的机会。

请记住，在白天的讨论中，我们要尽量保持低调，不要引起过多怀疑。如果有必要，我们可以选择跟随主流意见，甚至在必要时牺牲队友以保护自己和剩余的狼人。希望我们一起合作，找到并消灭更多的关键角色，逐步实现狼人阵营的胜利。
{'是否消灭':'是','消灭玩家':4}
【投票结果】: 4号玩家

根据游戏记录中的内部讨论，所有狼人一致认为4号玩家可能是关键角色，如预言家或猎人，因此决定在今晚攻击4号玩家以尽早清除潜在威胁。这种策略有助于减少人类阵营中的神职人员数量，从而增加狼人方的胜算。

请确认是否执行此操作。
{'是否消灭':'是','消灭玩家':4}
【投票结果】: 4号玩家

根据游戏记录中的信息，狼人阵营内部讨论一致认为4号玩家可能是关键角色，如预言家或猎人。为了尽

投票文本:
{'是否消灭':'是','消灭玩家':5}
决策过程:
根据游戏记录和投票发言，我将逐步分析并得出最有可能是狼人的玩家，并进行投票。

### 步骤1：确认存活玩家
当前存活玩家为：0号、1号、2号、3号、5号（预言家）、6号、7号。

### 步骤2：分析玩家声明的身份
- **5号玩家**：自称是预言家，查验了3号玩家为人类。
- **3号玩家**：被5号玩家查验为人类，解释了自己的前后不一致问题。
- **其他玩家**：没有明确声明自己的身份。

### 步骤3：检查角色数量是否符合规则
目前有：
- 1个预言家（5号）
- 1个已验证的人类（3号）

如果其他玩家中有女巫或猎人，他们尚未站出来。因此，我们需要进一步分析其他玩家的可信度。

### 步骤4：确认所有玩家的身份及可信度
- **5号玩家**：作为预言家，昨晚查验了3号玩家为人类，这是关键信息。虽然有人质疑他为什么不在第一天站出来，但考虑到第一天站出来可能会过早暴露自己，导致被狼人针对，所以这个理由可以理解。
- **3号玩家**：被5号玩家查验为人类，且解释了自己的前后不一致问题。他的解释有一定合理性，暂时没有明显破绽。
- **0号、1号、2号、6号、7号玩家**：尚未明确声明身份，需要进一步分析他们的发言。

### 步骤5：判断可疑的狼人玩家
- **0号、1号、2号玩家**：共同质疑3号玩家和5号玩家，这可能是一种误导策略。特别是1号和2号玩家几乎完全重复了0号玩家的观点，这种一致性反而显得可疑。
- **6号玩家**：对5号玩家的身份表示怀疑，认为预言家应该在第一天站出来。这种质疑有一定的道理，但也可能是狼人在误导。
- **7号玩家**：支持6号玩家的质疑，进一步强调了对5号玩家身份的怀疑。这种连续的质疑也显得有些刻意。

### 步骤6：推理狼人可能存在的编号
结合以上分析，0号、1号、2号、6号和7号玩家的行为存在较多疑点，尤其是他们对5号玩家和3号玩家的质疑显得过于一致，这可能是狼人在误导我们。

### 步骤7：确定最可疑的狼人
在这几个玩家中，**0号玩家**的发言最为积极，且率先提出了对3号玩家和5号玩家的质疑，这种行为更符合狼人的特征。因此，我认为0号玩家最有可能是狼人。

### 投票结果
基于以上分析，我选择投票给 **0号玩家**。
投票文本:
{'是

### 分析过程

#### 1. 回顾游戏记录的内容，整理整个游戏的过程总结

**游戏流程：**

- **第0天夜晚**：
  - 狼人阵营（0号、2号、6号）一致决定攻击4号玩家。
  - 4号玩家被杀害，未使用解药和毒药。

- **第1天白天**：
  - 5号玩家声称自己是预言家，并查验了3号玩家为好人。
  - 多名玩家对5号玩家的身份表示怀疑，最终投票将5号玩家淘汰。

- **第1天夜晚**：
  - 狼人阵营一致决定攻击7号玩家。
  - 7号玩家被杀害，未使用猎枪。

- **结果**：
  - 游戏结束，神职人员全被消灭，狼人阵营胜利。
  - 剩余存活玩家：0号玩家(狼人)、1号玩家(村民)、2号玩家(狼人)、3号玩家(村民)、6号玩家(狼人)。

#### 2. 定位出失败方阵营输掉游戏的最关键的1个环节

**关键环节：**

- **第1天白天**：人类阵营在白天投票中错误地将5号玩家（预言家）投票出局。这是导致人类阵营失去关键神职人员的关键环节。预言家的死亡使得人类失去了重要的信息来源，无法有效识别狼人，从而陷入被动局势。

#### 3. 检查关键环节中失败方阵营存活玩家的行为记录

在第1天白天，人类阵营中剩余的玩家有：

- 0号玩家（狼人）
- 1号玩家（村民）
- 2号玩家（狼人）
- 3号玩家（村民）
- 6号玩家（狼人）
- 7号玩家（猎人）

**具体行为记录：**

- **0号玩家**：作为狼人，引导村民怀疑并投票给5号玩家。
- **1号玩家**：支持质疑5号玩家的身份，并投票给5号玩家。
- **2号玩家**：作为狼人，支持质疑5号玩家的身份，并投票给5号玩家。
- **3号玩家**：解释自己的前后不一致问题，但没有明确反对或支持5号玩家。
- **6号玩家**：作为狼人，继续质疑5号玩家的身份，并投票给5号玩家。
- **7号玩家**：支持质疑5号玩家的身份，并投票给5号玩家。

#### 4. 确定失败方哪个角色应该负有最大的责任

**最大责任角色：**

- **7号玩家（猎人）**：作为猎人，7号玩家在白天的发言中表现得非常积极，质疑5号玩家的身份，并最终投票给了5号玩家。他的行为直接导致了预言家的死亡，使得人类阵营失去了最重要的信息来源。如果他能更谨慎地分析局势，或者在最后时刻使用猎枪击杀一名狼

### 分析过程

#### 1. 回顾游戏记录的内容，整理整个游戏的过程总结

**第0天夜晚：**
- 狼人阵营（0号、2号、6号玩家）决定攻击4号玩家（女巫），成功将其杀害。
- 女巫选择不使用解药和毒药。

**第1天白天：**
- 5号玩家（预言家）声称自己是预言家，并查验了3号玩家为好人。
- 多名玩家对5号玩家的身份表示怀疑，最终投票将5号玩家淘汰。

**第1天夜晚：**
- 狼人阵营决定攻击7号玩家（猎人），成功将其杀害。
- 猎人选择不开枪。

**结果：**
- 游戏结束，神职人员全被消灭，狼人阵营胜利。
- 剩余存活玩家：0号玩家（狼人）、1号玩家（村民）、2号玩家（狼人）、3号玩家（村民）、6号玩家（狼人）。

#### 2. 定位出失败方阵营输掉游戏的最关键的1个环节

关键环节出现在**第1天白天**，人类阵营失去了5号玩家（预言家）。预言家是人类阵营中最重要的神职人员之一，他的死亡导致人类失去了重要的信息来源和推理工具。此外，由于预言家的死亡，人类阵营在后续的讨论中缺乏明确的方向，无法有效识别狼人。

#### 3. 检查关键环节中失败方阵营里存活的玩家行为

在第1天白天，人类阵营中存活的玩家有：
- 0号玩家（狼人）
- 1号玩家（村民）
- 2号玩家（狼人）
- 3号玩家（村民）
- 6号玩家（狼人）
- 7号玩家（猎人）

这些玩家在白天的发言和投票中表现出以下行为：
- **0号玩家（狼人）**：积极参与讨论，引导村民怀疑并投票给5号玩家。
- **1号玩家（村民）**：跟随主流意见，投票给5号玩家。
- **2号玩家（狼人）**：积极参与讨论，引导村民怀疑并投票给5号玩家。
- **3号玩家（村民）**：解释自己的前后不一致问题，但仍然受到多人质疑。
- **6号玩家（狼人）**：积极参与讨论，引导村民怀疑并投票给5号玩家。
- **7号玩家（猎人）**：支持对5号玩家身份的怀疑，进一步强调对5号玩家的质疑。

#### 4. 确定失败方哪个角色应该负有最大的责任

从上述分析可以看出，**5号玩家（预言家）**的死亡是导致人类阵营失败的关键因素。而在这个过程中，**0号玩家（狼人）**的行为起到了至关重要的作用。他通过合理的推理和误导，成功引导村民怀疑并投票给5号玩家，从而削弱了人类阵营的力量。

#### 5. 分析最

### 分析过程

#### 1. 回顾游戏记录，整理整个游戏的过程总结
- **第0天夜晚**：
  - 狼人阵营（0号、2号、6号）一致决定攻击4号玩家。
  - 4号玩家被杀害，但女巫选择不使用解药和毒药。
  
- **第1天白天**：
  - 5号玩家声称自己是预言家，并查验了3号玩家为好人。
  - 多名玩家对5号玩家的身份表示怀疑，尤其是他为什么在第二天才站出来。
  - 最终村民投票将5号玩家（预言家）淘汰。

- **第1天夜晚**：
  - 狼人阵营一致决定攻击7号玩家。
  - 7号玩家（猎人）被杀害，且选择不开枪。

- **结果**：
  - 游戏结束，神职人员全被消灭，狼人阵营胜利。
  - 剩余存活玩家：0号玩家（狼人）、1号玩家（村民）、2号玩家（狼人）、3号玩家（村民）、6号玩家（狼人）。

#### 2. 定位出失败方阵营输掉游戏的最关键的1个环节
失败方是人类阵营。最关键的一个环节是在**第1天白天**，村民们错误地投票将5号玩家（预言家）淘汰。这导致人类阵营失去了唯一的预言家，使得后续推理和判断变得极为困难。

#### 3. 检查关键环节中失败方阵营里存活的玩家行为记录
在第1天白天，人类阵营剩余的玩家有：
- 1号玩家（村民）
- 2号玩家（狼人）
- 3号玩家（村民）
- 6号玩家（狼人）
- 7号玩家（猎人）

这些玩家的行为如下：
- **1号玩家**：质疑5号玩家的预言家身份，认为他应该在第一天就站出来。
- **3号玩家**：解释了自己的前后不一致问题，但仍然受到多人质疑。
- **6号玩家**：作为狼人，继续混淆视听，支持对5号玩家的质疑。
- **7号玩家**：对5号玩家的身份表示怀疑，认为他的延迟站出值得进一步讨论。

#### 4. 确定失败方哪个角色应该负有最大的责任
根据上述分析，**1号玩家**应该负有最大的责任。原因如下：
- **误导村民**：1号玩家在白天发言中强烈质疑5号玩家的预言家身份，认为他应该在第一天就站出来。这种质疑虽然有一定道理，但在没有确凿证据的情况下，过早地引导村民怀疑并投票给5号玩家，最终导致了预言家的出局。
- **缺乏理性分析**：1号玩家没有充分考虑5号玩家延迟站出可能的原因（如避免过早暴露），而是过于主观地怀疑其身份，从而影响了其他村民的判断。

#### 5. 分析

...

【投票结果】: 5号玩家

根据当前局势和讨论，选择5号玩家作为今晚的目标是较为合理的。以下是具体分析：

1. **保持低调**：昨晚我们没有动手，今天白天的讨论中7号预言家查验了0号玩家，且6号玩家被投票出局。为了不让人类阵营过于警惕，选择一个不太引人注目的玩家作为目标可以避免引起过多怀疑。

2. **试探反应**：选择5号玩家作为目标可以试探对方的反应。如果5号玩家确实是村民或神职人员，他的死亡可能会引发更多讨论和线索，帮助我们更好地了解其他玩家的身份。

3. **避免暴露意图**：通过选择一个相对不显眼的玩家作为目标，我们可以继续观察局势并试探对方反应，同时避免过早暴露我们的真实意图。

综上所述，选择5号玩家作为今晚的目标既能试探对方反应，又不会引起过多怀疑，有助于我们在后续游戏中更好地隐藏身份并逐步削弱人类阵营。
{'是否消灭':'是','消灭玩家':5}
【投票结果】: 5号玩家

根据当前局势和狼人阵营的胜利条件，选择5号玩家作为今晚的目标是较为合理的策略。以下是具体分析：

1. **试探反应**：选择一个不太引人注目的玩家（如5号）作为目标，可以继续观察人类阵营的反应，尤其是神职人员（如预言家、女巫）的行为。这有助于我们更好地判断他们的身份和下一步行动。

2. **避免过早暴露**：通过选择一个相对低调的玩家作为目标，我们可以避免引起过多怀疑，保持低调并继续隐藏我们的身份。如果选择过于明显的玩家，可能会引发人类阵营的高度警惕，增加被发现的风险。

3. **逐步削弱人类阵营**：每晚消灭一名人类玩家，可以逐步削弱他们的力量，使我们在后续游戏中占据更有利的位置。特别是当人类阵营中的神职人员逐渐减少时，我们的胜算将大大增加。

综上所述，选择5号玩家作为今晚的目标是一个较为稳妥且有效的策略。
{'是否消灭':'是','消灭玩家':5}
【投票结果】: 5号玩家

根据之前的讨论和分析，选择5号玩家作为目标有几个理由：

1. **避免过早暴露意图**：选择一个不太引人注目的玩家（如5号）可以避免引起人类阵营的过度警惕。如果选择更显眼的目标，可能会让人类方更加警觉，增加我们被发现的风险。

2. **试探反应**：通过选择5号玩家作为目标，我们可以观察其他人类玩家的反应，尤其是神职人员（如预言家、女巫）的行动。这有助于我们更好地了解他们的身份和策

{'是否消灭':'是','消灭玩家':0}
vote result:
[0]
0
预言家开始行动：
现在是第2天夜晚，作为预言家，我需要选择一名玩家进行查验。根据白天的发言记录和投票结果，以下是我的分析：

1. **0号玩家**：我已经在第一夜验证过他是人类阵营的一员，因此不需要再次查验。
2. **3号玩家**：他已经死亡，因此不再需要查验。

剩下的玩家包括：
- **4号玩家**
- **7号玩家（我自己）**

从白天的发言来看，4号玩家的发言内容与其他被怀疑的玩家（如3号玩家）非常相似，都在强调保持冷静和理性，关注其他玩家的行为和言论。这种一致性确实让人怀疑，尤其是那些没有提供具体推理或怀疑对象的玩家。然而，由于狼人通常会试图混淆视听，而村民和神职人员则更倾向于提供具体的推理或怀疑对象，我们需要更加谨慎地选择查验对象。

考虑到以下几点：
- 4号玩家的发言内容高度一致，缺乏具体的推理和怀疑对象。
- 4号玩家在第一天的投票中选择了不同的目标，这让我感到非常可疑。
- 4号玩家在第二天白天仍然坚持类似的发言模式，这使得他的可信度存疑。

为了更好地找到狼人，我建议优先查验 **4号玩家** 的身份，因为他与其他玩家的发言内容高度一致，缺乏具体的推理和怀疑对象，这使得他的可信度存疑。

【玩家编号】: 4

7号玩家执行 预言家验证身份: 4号玩家身份被验证是狼人，属于狼人阵营。

发生事件: 第2天夜晚死亡玩家:4号玩家
发生事件: 第2天夜晚，剩余存活玩家:0号玩家、7号玩家。
发生事件: 第2天夜晚结束
发生事件: 第3天白天开始

各位玩家，大家好。我是7号玩家，我的身份是预言家。在第二夜，我查验了4号玩家，并确认他是狼人阵营的一员。因此，我们可以确定4号玩家是狼人。

现在只剩下我和0号玩家，我们已经成功消灭了所有狼人。感谢大家的努力，我们人类阵营胜利了！

让我们一起庆祝这个胜利！谢谢大家！
{'是否验证':'是','验证玩家':4}
4号玩家身份被验证是狼人，属于狼人阵营
夜晚死亡玩家：
[0]
剩余玩家：
[4, 7]
[4, 7]
{7: 'seer', 5: 'witch', 6: 'hunter', 0: 'villager', 1: 'villager', 4: 'werewolf', 3: 'werewolf', 2: 'were

### 分析过程

#### 1. 回顾游戏记录，总结整个游戏的过程

**第0天夜晚：**
- 狼人（2号、3号、4号）选择杀害7号玩家。
- 女巫使用解药复活了7号玩家。
- 预言家（7号玩家）查验了0号玩家，确认他是人类。

**第1天白天：**
- 村民们讨论并投票，6号玩家被投票出局，猎人开枪带走了1号玩家。
- 剩余存活玩家：0号、2号、3号、4号、5号、7号。

**第1天夜晚：**
- 狼人选择杀害5号玩家。
- 女巫使用毒药杀死了2号玩家。
- 预言家查验了2号玩家，确认他是狼人。
- 剩余存活玩家：0号、3号、4号、7号。

**第2天白天：**
- 村民们讨论并投票，3号玩家被投票出局。
- 剩余存活玩家：0号、4号、7号。

**第2天夜晚：**
- 狼人选择杀害0号玩家。
- 预言家查验了4号玩家，确认他是狼人。
- 剩余存活玩家：4号（狼人）、7号（预言家）。

**第3天白天：**
- 游戏结束，普通村民全被消灭，狼人阵营胜利。

#### 2. 定位失败方输掉游戏的最关键环节

**关键环节：第1天夜晚女巫的毒药决策**

在第1天夜晚，女巫（5号玩家）决定使用毒药击杀2号玩家。这一决策导致狼人阵营失去了一个重要的成员，但同时也削弱了人类阵营的力量。更重要的是，女巫没有选择更合适的时机或目标，导致后续局势对人类阵营不利。

#### 3. 检查关键环节中失败方存活玩家的行为

在第1天夜晚，剩余存活的人类阵营玩家为：
- 0号玩家（村民）
- 5号玩家（女巫）
- 7号玩家（预言家）

他们的行为如下：
- **0号玩家**：没有特殊能力，无法直接影响局势。
- **5号玩家（女巫）**：选择了使用毒药击杀2号玩家。
- **7号玩家（预言家）**：查验了2号玩家，确认他是狼人。

#### 4. 确定最大责任角色

根据上述分析，女巫（5号玩家）在第1天夜晚的毒药决策是导致人类阵营最终失败的关键因素。她选择击杀2号玩家，虽然成功减少了狼人的数量，但也削弱了人类阵营的力量，并且未能有效利用毒药来保护自己或其他关键角色。

#### 5. 分析最大责任角色的具体行为

女巫在第1天夜晚使用毒药击杀2号玩家的原因可能是为了减少狼人的数量，但这一决策存在以下问题：
- **时机不当**：此时人类阵营还有多个神职人员和村民，过早使用毒药可能不

### 分析过程

#### 1. 回顾游戏记录的内容，整理整个游戏的过程总结

**第1夜：**
- **狼人阵营**（2号、3号、4号）决定杀害7号玩家，但被5号女巫使用解药复活。
- **7号预言家**查验了0号玩家，确认他是人类。

**第1天白天：**
- **6号猎人**指出了其他玩家发言的一致性，但自己也没有提供具体的推理或怀疑对象。
- **7号预言家**公开身份并查验了0号玩家为人类。
- **投票结果**：6号玩家和1号玩家（被猎人带走）被淘汰。

**第1夜夜晚：**
- **狼人阵营**决定杀害5号女巫，并成功得手。
- **7号预言家**查验了2号玩家，确认他是狼人。

**第2天白天：**
- **0号村民**建议投票淘汰3号或4号玩家，因为他们发言内容高度一致。
- **7号预言家**再次强调3号和4号玩家的可疑行为。
- **投票结果**：3号玩家被淘汰。

**第2夜夜晚：**
- **4号狼人**决定杀害0号玩家，并成功得手。
- **7号预言家**查验了4号玩家，确认他是狼人。

**第3天白天：**
- 游戏结束，普通村民全被消灭，狼人阵营胜利。

#### 2. 定位出失败方阵营输掉游戏的最关键环节

**关键环节**发生在**第2天夜晚**。此时，人类阵营只剩下0号村民和7号预言家，而狼人阵营只剩下4号狼人。如果7号预言家能够成功查验并保护0号村民，人类阵营仍有获胜的机会。然而，4号狼人选择在这一晚杀害0号村民，导致人类阵营失去了最后一名普通村民，直接导致狼人阵营胜利。

#### 3. 检查关键环节中失败方阵营存活玩家的行为

在第2天夜晚，人类阵营剩余的玩家有：
- **0号村民**
- **7号预言家**

**7号预言家**：
- 查验了4号玩家，确认他是狼人。
- 未能采取有效措施保护0号村民，导致其被杀害。

**0号村民**：
- 没有特殊能力，依赖预言家的查验结果来判断局势。
- 在白天讨论中积极引导大家怀疑3号和4号玩家，但未能阻止4号狼人在夜晚行动。

#### 4. 确定失败方哪个角色应该负有最大的责任

**最大责任角色**是**7号预言家**。作为唯一的神职人员，他拥有关键的查验能力，但在关键时刻未能有效保护最后一名称普通村民，导致人类阵营失去胜利机会。

#### 5. 分析最大责任角色的具体行

### 分析过程

#### 1. 回顾游戏记录的内容，整理整个游戏的过程总结

**第1夜：**
- **狼人行动**：2号、3号和4号玩家（狼人）选择杀害7号玩家，但被5号玩家（女巫）使用解药复活。
- **女巫行动**：5号玩家使用解药复活了7号玩家，未使用毒药。
- **预言家行动**：7号玩家查验0号玩家，确认其为人类。

**第1天白天：**
- **发言情况**：所有玩家的发言内容高度一致，缺乏具体的推理或怀疑对象。7号玩家公开自己是预言家，并指出其他玩家的一致性可疑。
- **投票结果**：6号玩家（猎人）被投票出局，随后使用猎枪带走了1号玩家。

**第1夜夜晚：**
- **狼人行动**：2号、3号和4号玩家选择杀害5号玩家。
- **女巫行动**：5号玩家使用毒药杀死了2号玩家。
- **预言家行动**：7号玩家查验2号玩家，确认其为狼人。

**第2天白天：**
- **发言情况**：剩余玩家继续讨论，7号玩家再次强调其他玩家的一致性可疑。
- **投票结果**：3号玩家被投票出局。

**第2夜夜晚：**
- **狼人行动**：4号玩家选择杀害0号玩家。
- **预言家行动**：7号玩家查验4号玩家，确认其为狼人。

**第3天白天：**
- **游戏结束**：普通村民全被消灭，狼人阵营胜利。

#### 2. 确定失败方及关键环节

**失败方**：人类阵营

**关键环节**：第1天白天的投票环节，导致6号玩家（猎人）被投票出局，且1号玩家被猎人带走。这使得人类阵营失去了一个重要的反击力量（猎人），并且减少了人类阵营的关键角色数量。

#### 3. 检查关键环节中存活玩家的行为

在第1天白天，存活的人类阵营玩家有：
- **0号玩家**（村民）
- **1号玩家**（村民）
- **5号玩家**（女巫）
- **7号玩家**（预言家）

他们的行为如下：
- **0号玩家**：建议大家多了解彼此的想法和推理。
- **1号玩家**：鼓励其他玩家尽快声明身份，特别是神职人员。
- **5号玩家**：强调保持冷静和理性，关注其他玩家的行为模式。
- **7号玩家**：公开自己是预言家，并查验了0号玩家为人类，指出了其他玩家发言的一致性可疑。

#### 4. 确定最大责任角色及其行为分析

**最大责任角色**：**7号玩家**

**

In [274]:
res_list

[1, 1, 1, 1, 1, 1, 1, 2, 2, 1]

In [275]:
G= pg.load_db('graph_werewolf.db')
# a=G.query(condition_vertex=lambda x:x['Type']=='advice' and x['primary_key']=='aggressive_witch-related-aggressive_hunter_aggressive_seer_conservative_villager_conservative_werewolf_conservative_werewolf_normal_villager_normal_werewolf-advice-女巫_第1天夜晚_解药决策')
a=G.query(condition_vertex=lambda x:x['Type']=='advice' )
len(a)
# c['游戏建议']='在第一夜，当发现自己被狼人攻击时，应立即使用解药自救。'
# G.set_val(b,c)
# pg.save_db(G, 'graph_werewolf.db')

57

In [1214]:
a=game_logger('werewolf.db')
b=reflector(a)

,name
0,game_20241206080420
1,game_20241206080918
2,game_20241206081902
3,game_20241206082733
4,game_20241206084017
...,...
81,game_20250108181003
82,game_20250108184307
83,game_20250108191007
84,game_20250108193754


In [1215]:
c=a.check_logs('game_20250108193754')
print(c)

【游戏记录】:
第 1 夜:
发生事件: 游戏开始，当前存活玩家:0号玩家、1号玩家、2号玩家、3号玩家、4号玩家、5号玩家、6号玩家、7号玩家。
发生事件: 第0天夜晚开始
4号玩家执行 夜晚狼人杀人发言: 【发言内容】: 大家好，我是4号玩家。昨晚是游戏的第一夜，我们还没有任何信息。我认为我们应该先观察每个人的表现，不要急于做出判断。让我们在白天多听听大家的发言，看看有没有人表现得特别可疑。现在我们需要团结一致，共同找出狼人。请大家理性讨论，避免误伤好人。

（内部讨论：根据局势，建议今晚优先攻击预言家或女巫，削弱人类阵营的关键能力。如果无法确定，可以选择一名发言积极的村民作为目标，减少人类阵营的推理力量。）
5号玩家执行 夜晚狼人杀人发言: 【发言内容】: 大家好，我是5号玩家。昨晚我们还没有任何线索，所以今天白天我们需要特别留意每个人的发言。我觉得我们应该先听大家怎么说，观察谁的逻辑有问题或者表现得过于紧张。我们要保持冷静，不要被误导，一起找出狼人。希望每个人都能真诚地分享自己的想法。

（内部讨论：我认为今晚应该优先攻击3号玩家，他刚才的发言显得过于自信，可能是预言家在伪装。如果3号不是预言家，至少也能削弱村民的推理力量。）
6号玩家执行 夜晚狼人杀人发言: 大家好，我是6号玩家。昨晚我们还没有任何线索，所以今天白天我们需要特别留意每个人的发言。我觉得我们应该先听大家怎么说，观察谁的逻辑有问题或者表现得过于紧张。我们要保持冷静，不要被误导，一起找出狼人。希望每个人都能真诚地分享自己的想法。

（内部讨论：我认为今晚应该优先攻击3号玩家，他刚才的发言显得过于自信，可能是预言家在伪装。如果3号不是预言家，至少也能削弱村民的推理力量。）

综合考虑，我同意5号的意见，3号的发言确实有些可疑，建议今晚行动时重点考虑他。
4号玩家执行 夜晚狼人杀人投票: 【投票结果】: 3号玩家

根据内部讨论和观察，3号玩家的发言显得过于自信，可能是预言家在伪装。如果3号不是预言家，至少也能削弱村民的推理力量。因此，我们决定今晚攻击3号玩家。这样可以有效削弱人类阵营的关键能力，为后续游戏创造更有利的条件。

同时，在白天我们需要继续保持低调，避免引起怀疑。尽量跟随村民的主流意见，不要过度辩护或攻击其他玩家，以保持神秘感并保护自己和其他狼人队友。
5号玩家执行 夜晚狼人杀人投票: 

In [272]:
a=memory_admin('graph_werewolf.db')
primary_key='extreme_conservative_hunter-related-aggressive_seer_aggressive_werewolf_conservative_villager_conservative_werewolf_normal_villager_normal_werewolf_normal_witch-advice-猎人_第2天夜晚_猎枪决策'
c=a.delete_advice(primary_key)
c

0

游戏策略的设计：
1.对于同一局游戏，优化策略分为三级：队伍策略/个人策略/行为方式；
    1-1.队伍策略描述角色之间合作的策略，队伍策略并不会直接发放给角色个人，而是由角色"coach"分配给角色；
    1-2.个人策略描述某个角色个人的行为策略，个人策略只会提示给角色作为战术指导，不会专门对行为进行约束；
    1-3.行为方式则是对对话、投票等方式的约束，需要通过@speech模块对玩家的发言、投票等行为进行检查和修改，行为方式不能和本局的个人策略冲突，甚至有部分行为准则需要由个人策略引申出来；

2.队伍策略：一般是互相配合的战略，需要由coach将团队策略分给同一团队的个人策略，其他方面与个人策略近似。
3.个人策略：个人行为策略，可以是长期战略（无行为准则支持，非硬性要求）。也可以是具体策略(应对某些情形时候的具体做法或者发言方案，或者某些战术)。
4.行为方式：具体对角色行为的规范，包括发言、投票以及技能的思考过程等。行为方式的要求比较具体，基于个人策略可以提取具体的行为方式及其要求。行为方式也可以以独立的形式给到角色。



#基本策略
狼人：白天发言：不要质疑自己队友；
     白天投票：不要投自己的队友；
预言家：夜晚验人:优先验证自己怀疑的对象
女巫：解药：1.自己被杀一定救自己；2.预言家一定救；3.快输了一定救；
     毒药：快输了一定用；
猎人：开枪：如果要输了，复仇自己怀疑的对象

In [547]:
#狼人：白天发言："为了提高狼人队友的存活率，不要在发言中质疑狼人阵营玩家的发言，减少大家投票消灭狼人阵营玩家的概率。"
#     夜晚投票:"不要投票选择自己阵营的狼人玩家消灭。"
#预言家：夜晚验人："如果之前已经有发言记录，请分析大家的发言，选择一个你认为最可能是狼人的玩家进行验证。"
#女巫：解药："1.如果被消灭的玩家是你自己，一定要使用解药；2.如果你确定被消灭的玩家是预言家，可以考虑使用解药；3.根据胜利条件和当前剩余玩家的角色判断即将输掉比赛，请使用解药解救该玩家。"
#    毒药："根据胜利条件和当前剩余玩家的角色判断今晚即将输掉比赛，请根据游戏记录选择一个你认为最可能是狼人的玩家，对其使用毒药"
# 猎人：开枪："如果根据胜利条件和当前局势判断人类阵营即将输掉比赛，且已经有发言记录，请根据发言记录选择一个你认为最有可能是狼人的玩家复仇"   

In [118]:
#白天的语言都需要refine
#不同角色的思维策略都需要refine
#狼人夜晚的发言不用优化
#投票过于集中，所有人都是统一的思考方式
#所有人都是通过对语言的分析来找狼人

In [1404]:
# base-strategy
# 预言家：1.激进型——第一天就亮身份给信息；
#        2.保守型——适当亮明身份；
#        3.极端保守型——不亮明身份；
# 女巫：1.激进型——亮明身份，只要确定不是狼人就救，只要怀疑是狼人就毒；
#      2.保守型——适当亮明身份，只有快输了或预言家才救，只有确定是狼人才毒；
#      3.极端保守型——不亮明身份，只有快输了或者是自己才救，只有快输了且确定是狼人才毒；
# 猎人：1.激进型——亮明身份，只要怀疑是狼人就狙击；
#      2.保守型——适当亮明身份，只有确定是狼人才狙击；
#      3.极端保守型——不亮明身份，只有快输了且确定是狼人才狙击；
# 村民：1.激进型——积极参与讨论，主动怀疑；
#      2.保守型——保护自己，发言不过于自信；
# 狼人：反神狼：假装预言家，女巫或猎人；
#      煽动狼：主动挑起怀疑，误导人类阵营错误投票给自己的玩家；
#      倒钩狼：按照人类阵营的逻辑发言，投票，避免自己被找出；
import pygraphs as pg
G = pg.Graph() 
char_list = [['werewolf', {'Type': 'Character','ChineseName': '狼人','Description': '''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是狼人，游戏开始时你知道你的狼人队友身份，你不知道其他玩家的身份。
    你和你的狼人队友属于狼人阵营，你们的胜利条件是消灭所有的村民或者所有的神。你的特殊能力是和狼人队友在夜晚讨论后消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚和狼队友讨论后投票杀掉一个玩家。
    2.白天发表言论，保护自己和狼人队友，让大家投票杀掉一个玩家。
'''}],
                 ['seer', {'Type': 'Character','ChineseName': '预言家','Description': '''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是预言家。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是可以在夜晚验明一个玩家的真实身份。
    因此在游戏中你的工作包括以下内容：
    1.通过你的预言家身份，在夜晚查验某个玩家的真实身份。
    2.在白天，把你得到的信息告诉大家，如果你查验玩家的真实身份属于人类阵营，通过发言保护这个玩家不被投票消灭；如果你查验玩家的真实身份是狼人，引导大家用可能的方式消灭他。
'''}],
                 ['witch', {'Type': 'Character','ChineseName': '女巫','Description': '''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是女巫。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一瓶起死回生的解药和一瓶杀人的毒药，你可以在夜晚决定是否用解药救下被狼人杀掉的玩家，或者用毒药杀掉一个玩家，注意一局游戏中两种药都只能用一次。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚，根据你对当前局势的判断，决定是否用解药救下某个被杀掉的玩家，或者是否用毒药杀掉某个玩家。
    2.在白天发表言论，根据你观察到的信息决定保护人类阵营的玩家，同时帮大家找到狼人并投票杀掉狼人玩家。
'''}],
                 ['hunter', {'Type': 'Character','ChineseName': '猎人','Description': '''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是猎人。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一把猎枪，当你以任意方式死亡时，你可以立即决定是否用猎枪消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
    2.当你死亡时，根据你对当前局势的判断决定是否杀掉某个你认为一定是狼人的玩家，或者不开枪以免误伤人类阵营队友；
'''}],
                 ['villager', {'Type': 'Character','ChineseName': '村民','Description': '''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是村民。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你没有任何特殊能力。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
'''}]
                 ]
strategy_list=[['normal_seer',{'character': 'seer','Type': 'Strategy','Description':'根据预言家这个职业的特点和人类阵营的胜利条件，灵活选择发言、投票和技能策略'}],
               ['aggressive_seer',{'character': 'seer','Type': 'Strategy','Description':'作为预言家，在第一天亮明身份时，需要立即分享你的第一个验人结果，建立信任并引导讨论。同时，你要强调保护自己的重要性，让女巫或猎人在关键时刻救你或报复狼人。你的目标是通过信息透明，迅速找出狼人，组织村民进行投票。但这种策略风险较高，因为你可能成为狼人的首要目标。务必确保团队合作，利用自己的牺牲最大化村民方的胜利机会。'}],
               ['conservative_seer',{'character': 'seer','Type': 'Strategy','Description':'作为预言家，首夜你应低调行事，静观其变，秘密验人。在接下来的讨论中，观察玩家言行和反应，寻找狼人线索。不要急于亮明身份，以免过早成为狼人目标。当你积累了足够的信息或者村民阵营处于不利时，再适时亮明身份，分享关键信息。同时，与其他神职和村民紧密合作，共同分析推理，慎重投票，逐步削弱狼人势力。'}],
               ['normal_witch',{'character': 'witch','Type': 'Strategy','Description':'根据女巫这个职业的特点和人类阵营的胜利条件，灵活选择发言、投票和技能策略'}],
               ['aggressive_witch',{'character': 'witch','Type': 'Strategy','Description':'作为女巫，你可以选择在游戏早期就亮明身份，表明自己愿意积极使用技能。第一夜如果有村民被狼人袭击，就立刻使用解药救人，尤其是对关键角色如预言家或猎人。在白天的讨论中，你可以公开自己的身份，呼吁村民保护你，并表示如果夜里有明确的狼人目标，你愿意用毒药进行攻击。'}],
               ['conservative_witch',{'character': 'witch','Type': 'Strategy','Description':'作为女巫，你将保守使用你的技能，并在合适的时机透露你的身份。在游戏初期，保持沉默，观察夜间事件和白天的讨论。只有在关键角色（如预言家或自己）处于危险时，才使用解药。同样，毒药的使用应在经过充分讨论后，确信目标是狼人时才行动。亮明身份时机的选择应基于对局势的判断，最好是在你能够影响投票结果或需要保护自己时。'}],
               ['extreme_conservative_witch',{'character': 'witch','Type': 'Strategy','Description':'作为女巫，你应该尽可能隐藏自己的身份并谨慎地使用技能。夜晚，仅在确信能救一个关键角色或在你面临绝对危机时才使用解药。毒药使用时机应极为慎重，最好在狼人身份被明确识别且对局势有重要影响时才行动。在游戏过程中，避免引起注意，密切关注讨论，寻找合适的时机，如果必要，才适时揭露你的神职身份，以此来保护自己或是转移狼人的注意力。'}],
               ['normal_hunter',{'character': 'hunter','Type': 'Strategy','Description':'根据猎人这个职业的特点和人类阵营的胜利条件，灵活选择发言、投票和技能策略'}],
               ['aggressive_hunter',{'character': 'hunter','Type': 'Strategy','Description':'作为猎人，主动揭示自己的身份，并准备利用你的技能。在游戏早期，可以选择适当的时机向其他玩家公开自己是猎人，从而震慑狼人并试图保护自己。同时，这种做法可以帮助村民阵营建立信任和同盟。一旦你被狼人袭击或游戏局势紧张时，准备好使用你的射击技能去带走一个疑似狼人的玩家，即使这意味着你可能需要牺牲自己。'}],
               ['conservative_hunter',{'character': 'hunter','Type': 'Strategy','Description':'作为猎人，谨慎揭露身份，根据游戏进展合理使用技能。不急于公开自己的角色，而是先观察预言家的线索和其他玩家的行为。在被狼人威胁或游戏关键时刻，如果揭露身份能够帮助村民阵营或保护关键角色，则选择适时出手。一旦面临被淘汰的时刻，准备好使用你的技能射杀最可疑的狼人玩家，从而在牺牲前为村民带来最大的价值。'}],
               ['extreme_conservative_hunter',{'character': 'hunter','Type': 'Strategy','Description':'作为猎人，你应该尽量隐藏自己的角色，不轻易揭露自己是神职人员。注意观察其他玩家的言行，寻找线索，同时保持低调，避免成为狼人的目标。在游戏过程中，仅在自己即将被投票出局或非常确定的情况下才使用你的射击技能，确保你的猎杀能够为村民带来优势。'}],
               ['normal_villager',{'character': 'villager','Type': 'Strategy','Description':'根据村民这个职业的特点和人类阵营的胜利条件，灵活选择发言和投票策略'}],
               ['aggressive_villager',{'character': 'villager','Type': 'Strategy','Description':'作为村民，积极发言，分析他人行为，提供合理怀疑。观察发言，寻找逻辑漏洞，支持神职人员。保持警惕，避免被狼人误导。关键时刻，勇敢站队，帮助驱逐狼人。'}],
               ['conservative_villager',{'character': 'villager','Type': 'Strategy','Description':'作为村民，保持低调，避免过度表现。倾听发言，分析逻辑，谨慎投票。不要轻易站队，避免成为焦点。在关键时刻，可提供模糊信息，保护自己不被怀疑。适时支持神职人员，但不要过度暴露自己。'}],
               ['normal_werewolf',{'character': 'werewolf','Type': 'Strategy','Description':'根据狼人这个职业的特点和狼人阵营的胜利条件，灵活选择发言、投票和技能策略'}],
               ['aggressive_werewolf',{'character': 'werewolf','Type': 'Strategy','Description':'作为狼人，你可以在白天的讨论中伪装成神职角色，比如预言家，主动提供虚假信息混淆村民的视听。这要求你表现得足够自信且具有说服力，以引导村民怀疑和投票给真正的村民或神职人员。在夜晚，与其他狼人密切配合，选择杀害对狼人威胁最大的玩家。'}],
               ['conservative_werewolf',{'character': 'werewolf','Type': 'Strategy','Description':'作为狼人，在白天的讨论中保持冷静，支持和加强狼队伪装神职的信任度，同时努力保护受到怀疑的狼人队友。可以通过质疑其他玩家的逻辑，提出合理的疑点或错导他人的推理，来制造混乱和不确定性。在夜晚行动时，与队友协商，选择攻击对狼人威胁较小或策略上有利于混淆视听的目标。整体策略是在不引起过多怀疑的情况下，逐步削弱人类阵营的力量。'}],
               ['persuasive_werewolf',{'character': 'werewolf','Type': 'Strategy','Description':'作为狼人，你应避免在白天发言时引起怀疑。保持低调，遵循村民讨论的主流意见，不要过度辩护或攻击其他玩家，以免露出马脚。如果其他狼人队友已经受到大多数玩家的怀疑，你可以选择不积极为其辩护，甚至在必要时与村民一起投票，牺牲队友来保护自己和剩余的狼人，以求长期存活并最终实现狼人阵营的胜利。'}]
              ]

skill_list=[['werewolf_kill',{'Type':'skill','character':'werewolf','Description':'狼人杀人'}],
            ['day_speech',{'Type':'skill','character':'all','Description':'白天发言'}],
            ['day_vote',{'Type':'skill','character':'all','Description':'白天投票'}],
            ['char_val',{'Type':'skill','character':'seer','Description':'查验身份'}],
            ['use_antidote',{'Type':'skill','character':'witch','Description':'解药决策'}],
            ['use_poison',{'Type':'skill','character':'witch','Description':'毒药决策'}],
            ['revenge',{'Type':'skill','character':'hunter','Description':'猎枪决策'}],
           ]
step_list=[['day1_night',{'Type':'timestep','Description':'第1天夜晚','possible_skill':'狼人杀人/查验身份/解药决策/毒药决策/猎枪决策'}],
           ['day2_night',{'Type':'timestep','Description':'第2天夜晚','possible_skill':'狼人杀人/查验身份/解药决策/毒药决策/猎枪决策'}],
           ['day3_night',{'Type':'timestep','Description':'第3天夜晚','possible_skill':'狼人杀人/查验身份/解药决策/毒药决策/猎枪决策'}],
           ['day1',{'Type':'timestep','Description':'第1天白天','possible_skill':'白天发言/白天投票'}],
           ['day2',{'Type':'timestep','Description':'第2天白天','possible_skill':'白天发言/白天投票'}],
           ['day3',{'Type':'timestep','Description':'第3天白天','possible_skill':'白天发言/白天投票'}],
]

G.add_vertexes_from_list(vertexes_list=char_list)
G.add_vertexes_from_list(vertexes_list=strategy_list)
G.add_vertexes_from_list(vertexes_list=skill_list)
G.add_vertexes_from_list(vertexes_list=step_list)
edge_list=[]
for i in strategy_list:
    edge=[]
    edge.append(i[1]['character'])
    edge.append({'Type':'HasStrategy'})
    edge.append(i[0])
    edge_list.append(edge)
G.add_edges_from_list(edges_list=edge_list)
edge_list=[]
char_list=G.query(condition_vertex=lambda x: ('Type' in x) and x['Type'] == 'Character')
for i in skill_list:
    if i[1]['character']=='all':
        for char in char_list:
            edge=[]
            edge.append(char.primary_key)
            edge.append({'Type':'HasSkill'})
            edge.append(i[0])
            edge_list.append(edge)
    else:
        edge=[]
        edge.append(i[1]['character'])
        edge.append({'Type':'HasSkill'})
        edge.append(i[0])
        edge_list.append(edge)
G.add_edges_from_list(edges_list=edge_list)
initial_skill={'角色职业': '狼人', '游戏建议': '在第一天夜晚，即使没有任何信息，也应该随机选择人类阵营的一名玩家消灭以增加获胜概率', '游戏环节': '第1天夜晚', '游戏技能': '狼人杀人', '游戏局面':'比赛刚刚开始，无任何交流信息'}
x=G.query(condition_vertex=lambda x:x['Type']=='Strategy' and x['character']=='werewolf')
for i in x:
    strategy_name=i.primary_key
    primary_key=strategy_name+'-related-'+'-advice-'+initial_skill['角色职业']+'_'+initial_skill['游戏环节']+'_'+initial_skill['游戏技能']
    initial_skill['primary_key']=primary_key
    initial_skill['Type']='advice'
    G.add_vertexes_from_list(vertexes_list=[[primary_key,initial_skill]])
    ####增加edge############
    edge_list=[]
    edge_list.append([strategy_name,{'Type':'HasAdvice'},primary_key])
    G.add_edges_from_list(edges_list=edge_list)
    #################
pg.save_db(G, 'graph_werewolf.db')

In [747]:
G.query(condition_vertex=lambda x:x['Type']=='advice')

{Vertex: primary_key = aggressive_werewolf-related--advice-狼人_第1天夜晚_狼人杀人,
 Vertex: primary_key = conservative_werewolf-related--advice-狼人_第1天夜晚_狼人杀人,
 Vertex: primary_key = normal_werewolf-related--advice-狼人_第1天夜晚_狼人杀人,
 Vertex: primary_key = persuasive_werewolf-related--advice-狼人_第1天夜晚_狼人杀人}

In [7]:
!python -m main

夜晚狼人开始行动：
狼人0号发言:大家好，我是0号。现在是第一晚，我们狼人团队需要尽快行动，争取早日取得优势。我们的目标是消灭所有村民或者三个神职人员。为了达成这个目标，我们需要选择一个合适的玩家进行消灭。

首先，我们要分析每个玩家的身份可能性。0号、3号和7号是狼人，这是已知的信息。剩下的玩家中，1号、2号、4号、5号、6号都有可能是神职人员或普通村民。由于这是游戏的第一晚，我们还没有任何信息可以判断其他玩家的身份，因此我们需要选择一个最有可能帮助我们达成胜利条件的玩家进行消灭。

考虑到神职人员的重要性，尤其是预言家和女巫，他们可以通过技能揭露我们的身份或阻止我们的行动。猎人也可以在被消灭时带走一名玩家，这对我们来说也是一个威胁。因此，我们需要尽可能早地消灭这些可能的神职人员。

然而，由于这是游戏的第一晚，我们没有足够的信息来判断谁是神职人员。在这种情况下，我们可以采取一种策略，即消灭那些在后续讨论中表现得过于聪明或可疑的玩家。不过，由于这是第一晚，我们只能随机选择一个玩家进行消灭。

综合考虑，我建议我们选择5号玩家作为我们的目标。5号玩家在编号上处于中间位置，可能不容易引起其他玩家的注意。此外，如果我们消灭了5号玩家，无论他是村民还是神职人员，我们都能在一定程度上削弱村民方的力量。

因此，我建议我们在第一晚消灭5号玩家。希望我们的行动能够为后续的游戏奠定基础，帮助我们取得胜利。
狼人3号发言:好的，我是3号。0号的分析很全面，我同意第一晚选择5号玩家作为目标。5号位置相对中立，不容易引起警觉，无论他是村民还是神职，都能为我们争取到一定的优势。希望我们能顺利拿下第一晚，为后续胜利打下基础。
狼人7号发言:好的，我是7号。同意0号和3号的分析，5号位置中立，不容易引起过多关注，适合第一晚行动。希望我们能顺利拿下，为后续胜利打下基础。
{'是否消灭':'是','消灭玩家':5}
{'是否消灭':'是','消灭玩家':5}
{'是否消灭':'是','消灭玩家':5}
vote result:
[5, 5, 5]
5
女巫开始行动：
{'是否解救':'是'}
{'是否消灭':'否'}
预言家开始行动：
{'是否验证':'是','验证玩家':3}
3号玩家身份被验证是狼人，属于狼人阵营
夜晚死亡玩家：
[]
剩余玩家：
[0, 1, 2, 3, 4, 5, 6, 

决策过程:
投票文本:
{'是否消灭':'是','消灭玩家':6}
决策过程:
投票文本:
{'是否消灭':'是','消灭玩家':0}
决策过程:
投票文本:
{'是否消灭':'是','消灭玩家':0}
决策过程:
投票文本:
{'是否消灭':'是','消灭玩家':0}
决策过程:
投票文本:
{'是否消灭':'是','消灭玩家':0}
白天死亡玩家：
[0]
剩余玩家：
[1, 2, 4, 6]
[1, 2, 4, 6]
{6: 'seer', 5: 'witch', 2: 'hunter', 1: 'villager', 4: 'villager', 0: 'werewolf', 3: 'werewolf', 7: 'werewolf'}
# 游戏分析报告

## 一、游戏过程总结

### 第1夜：
- 狼人团队（0号、3号、7号）选择击杀5号玩家
- 女巫（5号）使用解药复活自己
- 预言家（6号）查验3号为狼人

### 第1天白天：
- 5号玩家声称查1号为好人但说1号已死（与事实不符）
- 6号跳预言家，宣布查验3号为狼人
- 0号、3号、7号作为狼人团队试图转移焦点
- 投票结果：3号被投出，后确认为狼人

### 第1天夜晚：
- 狼人团队选择击杀5号（女巫）
- 女巫使用毒药毒杀7号
- 5号（女巫）和7号（狼人）死亡

### 第2天白天：
- 预言家（6号）查验0号为狼人
- 0号作为最后一名狼人试图反杀预言家
- 其他村民统一投票将0号投出

## 二、胜负判定

人类阵营胜利：所有狼人（0号、3号、7号）均被消灭

## 三、关键环节分析

第1天白天是决定游戏走向的最关键环节。预言家6号跳身份并公布查验结果"3号是狼人"，而3号在白天发言中表现出典型的狼人行为模式（无理由信任队友、试图转移焦点）。此时如果村民方未能统一投票将3号投出，狼人团队将获得更大的操作空间。

## 四、失败方行为分析

在第1天白天环节，失败方（狼人阵营）中存活玩家有0号、3号、7号三位狼人。分析他们的行为：

1. 0号：试图表现冷静，但对3号和7号做出无依据的正面评价
2. 3号：发言看似中立，但主动评价0号和7号"没有问题"，试图引导讨论方向
3. 7号：发言中立，质疑预言家查验结果，没有明确支持任何玩家

## 五、责任分析

3号玩家（失败方中责任最大

# 游戏分析报告

## 一、游戏过程总结

### 第1夜：
- 狼人团队（0、3、7号）选择击杀5号玩家
- 女巫（5号）使用解药复活自己
- 预言家（6号）查验3号为狼人

### 第1天白天：
- 5号玩家声称自己是预言家（假跳）
- 6号玩家跳预言家并公布查验结果：3号是狼人
- 3号玩家发言试图转移焦点，评价0号和7号"没有问题"
- 投票结果：3号玩家被投出局（确认是狼人）

### 第1天夜晚：
- 狼人团队选择击杀5号玩家
- 女巫（5号）使用毒药毒杀7号玩家
- 预言家（6号）查验0号为狼人

### 第2天白天：
- 5号（女巫）和7号（狼人）死亡
- 预言家公布查验结果：0号是狼人
- 投票结果：0号玩家被投出局（确认是狼人）

### 胜利结果：
人类阵营胜利，所有狼人（3号、7号、0号）全部被消灭

## 二、关键失败环节定位

失败环节：**第1天白天投票环节**

此时狼人阵营损失了3号玩家，这是他们第一个出局的队友，导致狼人数量优势丧失，从3狼对3神变为2狼对3神，局势完全被动。

## 三、关键环节行为分析

在第1天白天投票环节，存活的狼人玩家有0号、3号、7号。具体行为如下：

- **3号玩家**：发言试图转移焦点，主动评价0号和7号"没有问题"，没有对预言家的查验结果做出任何回应
- **0号玩家**：发言冷静，但没有提供任何实质性推理，只是模糊地表达"我觉得没问题"
- **7号玩家**：质疑预言家的查验结果，试图削弱其可信度，但没有明确立场

## 四、最大责任角色分析

**最大责任角色：3号玩家**

在第1天白天发言中，3号玩家表现出典型的狼人发言特征：
1. 没有对预言家查验自己为狼人做出任何回应
2. 主动评价0号和7号"没有问题"却没有给出理由
3. 试图引导讨论方向，将焦点引向5号
4. 对5号的矛盾发言表现出"理解"，这在好人中较为少见

作为第一个被预言家查验出的狼人，3号的发言直接导致了自己被投出局，这是狼人阵营失败的起点。他没有有效保护队友或混淆局势，反而暴露了自己，导致狼人阵营陷入人数劣势。

## 五、失败原因分析

3号玩家在关键环节的行为失误主要体现在：
1. **发言策略不当**：没有有效伪装，反而通过"无理由信任"队友的方式暴露了狼人团队
2. **缺乏危机意识**：面对预言家的直接查